In [1]:
import pandas as pd
import json
import numpy as np

# what i want for a repeatable process:
1. try refetching some lyrics (potentialy with a new strategy)
2. save the ones that worked to disk
3. (repeat)


In [2]:
songs_df = pd.read_csv('./data/1 DONE RIGHT OUTPUT unique songs.csv')
songs_df

,song_id,chart_position,chart_debut,song,performer,artist_array,generic_genre
0,#9 DreamJohn Lennon,9,1974-12-21,#9 Dream,John Lennon,['John Lennon'],uncategorized
1,'03 Bonnie & ClydeJay-Z Featuring Beyonce Knowles,4,2002-10-26,'03 Bonnie & Clyde,Jay-Z Featuring Beyonce Knowles,"['Jay-Z', 'Beyonce Knowles']",rock
2,'65 Love AffairPaul Davis,6,1982-02-27,'65 Love Affair,Paul Davis,['Paul Davis'],hip hop
3,('til) I Kissed YouThe Everly Brothers,4,1959-08-15,('til) I Kissed You,The Everly Brothers,['The Everly Brothers'],country
4,(Can't Live Without Your) Love And AffectionNe...,1,1990-07-07,(Can't Live Without Your) Love And Affection,Nelson,['Nelson'],uncategorized
...,...,...,...,...,...,...,...
5061,everything i wantedBillie Eilish,8,2019-11-23,everything i wanted,Billie Eilish,['Billie Eilish'],folk
5062,iSpyKYLE Featuring Lil Yachty,4,2017-01-14,iSpy,KYLE Featuring Lil Yachty,"['KYLE', 'Lil Yachty']",pop
5063,interludeJ. Cole,8,2021-05-22,interlude,J. Cole,['J. Cole'],NaN
5064,"my.lifeJ. Cole, 21 Savage & Morray",2,2021-05-29,my.life,"J. Cole, 21 Savage & Morray","['J. Cole', '21 Savage', 'Morray']",NaN


## Ok, new strategy: the original dataset is our source of truth (ie to do joins/merges off of). 

In [312]:
def save_lyrics(lyrics_subset, file_name):
    lyrics_subset.to_csv(f'./data/lyric-fetching-datasets/{file_name}.csv', index=False)
    return lyrics_subset

## Fetching functions

In [165]:
BANNED_CHARACTERS = ['[', ']', '"', "'", '+', ',', '.', '(', ')', '/', '\\']
def clean_it(s):
    cleaned = s
    for naughty_char in BANNED_CHARACTERS:
        cleaned = cleaned.replace(naughty_char, '')
    return cleaned.strip()

dirty = 'Whole Wide World (From "True Love"'
clean_it(dirty)

'Whole Wide World From True Love'

In [5]:
import lyricsgenius
GENIUS_CLIEN_API_KEY = '8ZiCNVQqbSF6NM0nwVnVv6Y0EBR7BsVQ9HaVgPdqJOZR1l6XlymR-pUUi6z3GHPx' # be nice :)
genius = lyricsgenius.Genius(GENIUS_CLIEN_API_KEY)

In [32]:
def search_song_chose_first_then_get_lyrics(song, performer):
    try: 
        print(f'{performer} {song}')
        print('CLEANED: ', clean_it(f'{performer} {song}'))
        songs = genius.search_songs(clean_it(f'{performer} {song}'))
        urls = [song['result']['url'] for song in songs['hits']]
        print(urls)
        return [genius.lyrics(song_url=urls[0]), urls]
    except:
        print(f'OOPS: {performer} {song}')

In [33]:
def fetch_lyrics_and_urls(songs_subset_df):
    return songs_subset_df.apply(
        lambda row: search_song_chose_first_then_get_lyrics(song=row['song'], performer=row['performer']),
        axis=1
    )

In [34]:
def convert_fetched_lyrics_to_df(fetched_lyrics_and_url_options_arrays, original_df):
    just_lyrics = [x and x[0] for x in fetched_lyrics_and_url_options_arrays]
    just_stringified_urls = [x and json.dumps(x[1]) for x in fetched_lyrics_and_url_options_arrays]
    save_to_disk = pd.DataFrame({
        'song_id': original_df.song_id, 'lyrics': just_lyrics, 'urls': just_stringified_urls
    })
    return save_to_disk

### 1-line demo

In [10]:
convert_fetched_lyrics_to_df(
    fetch_lyrics_and_urls(
        songs_df.head(1)
    ), 
    songs_df.head(1)
)

John Lennon #9 Dream
CLEANED:  John Lennon #9 Dream
['https://genius.com/John-lennon-9-dream-lyrics', 'https://genius.com/Rolling-stone-the-500-greatest-songs-of-all-time-2021-annotated', 'https://genius.com/John-lennon-oh-yoko-take-9-lyrics', 'https://genius.com/Genius-january-2021-album-release-calendar-annotated', 'https://genius.com/Pitchfork-the-200-best-songs-of-the-1970s-annotated', 'https://genius.com/Rock-genius-significant-dates-in-rock-history-postlet-helper-lyrics', 'https://genius.com/Wild-prxfits-shit-popped-off-lyrics', 'https://genius.com/Genius-dream-album-dream-album-g-eazy-by-slickk-lyrics', 'https://genius.com/Genius-dream-album-dream-album-paul-mccartney-by-vesuvius-annotated', 'https://genius.com/Salloum-true-story-lyrics']


,song_id,lyrics,urls
0,#9 DreamJohn Lennon,#9 Dream Lyrics[Verse 1]\nSo long ago\nWas it ...,"[""https://genius.com/John-lennon-9-dream-lyric..."


# OK, LET'S JUST GET ALL THE LYRICS WE *HAVE* TRIED FETCHING INTO ONE PLACE (work done previously, in this notebook, & in others)

In [11]:
LYRIC_FOLDER_PATH = './data/lyric-fetching-datasets'

### OK, this is the older fetched stuff, which we already fetched lyrics using (ie a few of these URLS are wrong, & we tried to get better ones, so we should overwrite with those new ones)

## ✅ 1. our big 4,600 row list of succesfully fetched lyrics (some may be wrong, but we'll correct that as we merge others in)

In [12]:
has_lyrics_df = pd.read_csv(f'{LYRIC_FOLDER_PATH}/{"has_lyrics"}.csv')
has_lyrics_df

,song_id,lyrics,urls
0,(I Can't Get No) SatisfactionThe Rolling Stones,(I Can’t Get No) Satisfaction Lyrics[Instrumen...,"[""https://genius.com/The-rolling-stones-i-cant..."
1,(I Just) Died In Your ArmsCutting Crew,(I Just) Died in Your Arms Lyrics[Intro]\nOh I...,"[""https://genius.com/Cutting-crew-i-just-died-..."
2,(I Know) I'm Losing YouThe Temptations,(I Know) I’m Losing You Lyrics[Intro]\nOoh\nYo...,"[""https://genius.com/The-temptations-i-know-im..."
3,(I've Had) The Time Of My LifeBill Medley & Je...,(I’ve Had) The Time of My Life Lyrics[Chorus: ...,"[""https://genius.com/Bill-medley-and-jennifer-..."
4,1 ThingAmerie,1 Thing Lyrics[Intro]\nWoo!\nUh\nWoo!\nNa-na-n...,"[""https://genius.com/Amerie-1-thing-lyrics"", ""..."
...,...,...,...
4638,abcdefuGAYLE,TranslationsTürkçeEspañolDeutschFrançais​abcde...,"[""https://genius.com/Gayle-abcdefu-lyrics"", ""h..."
4639,amariJ. Cole,"American Dream Lyrics[Intro: Jeezy]\n(Cryin', ...","[""https://genius.com/Jeezy-american-dream-lyri..."
4640,iSpyKYLE Featuring Lil Yachty,"I Spy, T Shirt, Isn’t She Lovely, & Swang Mash...","[""https://genius.com/Alex-aiono-i-spy-t-shirt-..."
4641,interludeJ. Cole,"Interlude (Cole World) LyricsI'm drivin', nigg...","[""https://genius.com/J-cole-interlude-cole-wor..."


In [36]:
has_lyrics_df.lyrics.dropna()

0       (I Can’t Get No) Satisfaction Lyrics[Instrumen...
1       (I Just) Died in Your Arms Lyrics[Intro]\nOh I...
2       (I Know) I’m Losing You Lyrics[Intro]\nOoh\nYo...
3       (I’ve Had) The Time of My Life Lyrics[Chorus: ...
4       1 Thing Lyrics[Intro]\nWoo!\nUh\nWoo!\nNa-na-n...
                              ...                        
4638    TranslationsTürkçeEspañolDeutschFrançais​abcde...
4639    American Dream Lyrics[Intro: Jeezy]\n(Cryin', ...
4640    I Spy, T Shirt, Isn’t She Lovely, & Swang Mash...
4641    Interlude (Cole World) LyricsI'm drivin', nigg...
4642    TranslationsРусскийPortuguêsDeutschItaliano​​m...
Name: lyrics, Length: 4643, dtype: object

## ✅ 2. 69 songs where we re-fetched them cuz the lyrics looked obviously wrong

In [13]:
better_url_refetched_lyrics = pd.read_csv(f'{LYRIC_FOLDER_PATH}/{"TODO we refetched approx 70 songs with a more likely URL, integrate them"}.csv')
better_url_refetched_lyrics


,song_id,lyrics,urls,likely_correct_url
0,6 Foot 7 FootLil Wayne Featuring Cory Gunz,TranslationsEnglish6 Foot 7 Foot Lyrics[Chorus...,"[""https://genius.com/Genius-users-best-lines-b...",https://genius.com/Lil-wayne-6-foot-7-foot-lyrics
1,679Fetty Wap Featuring Remy Boyz,679 Lyrics[Produced by Peoples]\n\n[Intro: Fet...,"[""https://genius.com/Spotify-new-music-tuesday...",https://genius.com/Fetty-wap-679-lyrics
2,ApologizeTimbaland Featuring OneRepublic,"Apologize Lyrics[Intro: Timbaland]\nAyy, ayy, ...","[""https://genius.com/Nvp-summer-playlist-2015-...",https://genius.com/Timbaland-apologize-lyrics
3,Bad And BoujeeMigos Featuring Lil Uzi Vert,TranslationsPortuguêsРусскийItalianoAzərbaycan...,"[""https://genius.com/Miguel-castillo-top-5-jam...",https://genius.com/Migos-bad-and-boujee-lyrics
4,BillionaireTravie McCoy Featuring Bruno Mars,Billionaire Lyrics[Pre-Chorus: Bruno Mars]\nI ...,"[""https://genius.com/Formerly-vinylzombie-mcr-...",https://genius.com/Travie-mccoy-billionaire-ly...
...,...,...,...,...
64,We Are Youngfun. Featuring Janelle Monae,We Are Young Lyrics[Verse 1: Nate Ruess]\nGive...,"[""https://genius.com/Adoseofbuckley-the-ten-wo...",https://genius.com/Fun-we-are-young-lyrics
65,We Found LoveRihanna Featuring Calvin Harris,TranslationsPortuguêsEnglishFrançaisWe Found L...,"[""https://genius.com/Npr-the-200-greatest-song...",https://genius.com/Rihanna-we-found-love-lyrics
66,We R Who We RKe$ha,TranslationsPortuguêsWe R Who We R Lyrics[Vers...,"[""https://genius.com/Adoseofbuckley-adoseofbuc...",https://genius.com/Kesha-we-r-who-we-r-lyrics
67,Young ForeverJay-Z + Mr. Hudson,Young Forever Lyrics[Intro: Mr. Hudson]\nLet's...,"[""https://genius.com/Genius-deutschland-die-li...",https://genius.com/Jay-z-young-forever-lyrics


In [14]:
(better_url_refetched_lyrics.lyrics.str.len() < 6000).describe()

count       69
unique       2
top       True
freq        58
Name: lyrics, dtype: object

### This is a great sign: there are no longer any obviously too long lyrics (except one, which I added to the instrumental list)

In [15]:
better_url_refetched_lyrics[
    better_url_refetched_lyrics.lyrics.str.len() > 6000
]

,song_id,lyrics,urls,likely_correct_url
3,Bad And BoujeeMigos Featuring Lil Uzi Vert,TranslationsPortuguêsРусскийItalianoAzərbaycan...,"[""https://genius.com/Miguel-castillo-top-5-jam...",https://genius.com/Migos-bad-and-boujee-lyrics
39,Mona LisaLil Wayne Featuring Kendrick Lamar,TranslationsTürkçeРусскийMona Lisa Lyrics[Intr...,"[""https://genius.com/Genius-users-musikalische...",https://genius.com/Lil-wayne-mona-lisa-lyrics
45,Only YouFranck Pourcel's French Fiddles,Grünt #15 Lyrics[Couplet 1 : Alpha Wann]\nPhil...,"[""https://genius.com/Edmund-burke-reflections-...",https://genius.com/Grunt-grunt-15-lyrics


### Ok, but a few songs need to be refetched:

In [16]:
better_url_refetched_lyrics[
    better_url_refetched_lyrics.lyrics.isna()
]

,song_id,lyrics,urls,likely_correct_url
11,Drop It Like It's HotSnoop Dogg Featuring Phar...,NaN,NaN,https://genius.com/Snoop-dogg-drop-it-like-its...
17,Flashdance...What A FeelingIrene Cara,NaN,NaN,https://genius.com/Irene-cara-flashdance-what-...
27,I Love ItIcona Pop Featuring Charli XCX,NaN,NaN,https://genius.com/Icona-pop-i-love-it-lyrics
33,Let Me Blow Ya MindEve Featuring Gwen Stefani,NaN,NaN,https://genius.com/Eve-let-me-blow-ya-mind-lyrics
40,Moves Like JaggerMaroon 5 Featuring Christina ...,NaN,NaN,https://genius.com/Maroon-5-moves-like-jagger-...
47,PayphoneMaroon 5 Featuring Wiz Khalifa,NaN,NaN,https://genius.com/Maroon-5-payphone-lyrics
50,Rather BeClean Bandit Featuring Jess Glynne,NaN,NaN,https://genius.com/Clean-bandit-rather-be-lyrics
63,UmbrellaRihanna Featuring Jay-Z,NaN,NaN,https://genius.com/Rihanna-umbrella-lyrics


In [41]:
def _get_song(url, urls):
    try: 
        print(url, '...')
        return [genius.lyrics(song_url=url), urls]
    except:
        print(f'OOPS: {url}')
        
def refetch_lyrics_and_urls(songs_subset_df):
    return songs_subset_df.apply(
        lambda row: _get_song(row['likely_correct_url'], row['urls']),
        axis=1
    )

once_again_with_better_url = refetch_lyrics_and_urls(
    better_url_refetched_lyrics[
        better_url_refetched_lyrics.lyrics.isna()
    ]
)
once_again_with_better_url

https://genius.com/Snoop-dogg-drop-it-like-its-hot-lyrics ...
https://genius.com/Irene-cara-flashdance-what-a-feeling-lyrics ...
https://genius.com/Icona-pop-i-love-it-lyrics ...
https://genius.com/Eve-let-me-blow-ya-mind-lyrics ...
https://genius.com/Maroon-5-moves-like-jagger-lyrics ...
https://genius.com/Maroon-5-payphone-lyrics ...
https://genius.com/Clean-bandit-rather-be-lyrics ...
https://genius.com/Rihanna-umbrella-lyrics ...


11    [TranslationsPortuguêsDrop It Like It’s Hot Ly...
17    [Flashdance... What a Feeling Lyrics[Intro]\nF...
27    [TranslationsPortuguêsI Love It Lyrics[Chorus:...
33    [Let Me Blow Ya Mind Lyrics[Intro: Eve]\nUh, u...
40    [Moves Like Jagger Lyrics[Intro: Adam Levine]\...
47    [TranslationsPortuguêsPayphone Lyrics[Chorus: ...
50    [TranslationsPortuguêsFrançaisRather Be Lyrics...
63    [Umbrella Lyrics[Intro: JAY-Z]\nUh-huh, uh-huh...
dtype: object

In [42]:
def convert_fetched_lyrics_to_df(fetched_lyrics_and_url_options_arrays, original_df):
    just_lyrics = [x and x[0] for x in fetched_lyrics_and_url_options_arrays]
    just_stringified_urls = [x and json.dumps(x[1]) for x in fetched_lyrics_and_url_options_arrays]
    save_to_disk = pd.DataFrame({
        'song_id': original_df.song_id, 'lyrics': just_lyrics, 'urls': just_stringified_urls, 
        'likely_correct_url': original_df.likely_correct_url
    })
    return save_to_disk

In [43]:
better_url_refetched_lyrics[
        better_url_refetched_lyrics.lyrics.isna()
    ]

,song_id,lyrics,urls,likely_correct_url
11,Drop It Like It's HotSnoop Dogg Featuring Phar...,NaN,NaN,https://genius.com/Snoop-dogg-drop-it-like-its...
17,Flashdance...What A FeelingIrene Cara,NaN,NaN,https://genius.com/Irene-cara-flashdance-what-...
27,I Love ItIcona Pop Featuring Charli XCX,NaN,NaN,https://genius.com/Icona-pop-i-love-it-lyrics
33,Let Me Blow Ya MindEve Featuring Gwen Stefani,NaN,NaN,https://genius.com/Eve-let-me-blow-ya-mind-lyrics
40,Moves Like JaggerMaroon 5 Featuring Christina ...,NaN,NaN,https://genius.com/Maroon-5-moves-like-jagger-...
47,PayphoneMaroon 5 Featuring Wiz Khalifa,NaN,NaN,https://genius.com/Maroon-5-payphone-lyrics
50,Rather BeClean Bandit Featuring Jess Glynne,NaN,NaN,https://genius.com/Clean-bandit-rather-be-lyrics
63,UmbrellaRihanna Featuring Jay-Z,NaN,NaN,https://genius.com/Rihanna-umbrella-lyrics


In [49]:
lol_the_had_to_refetch_a_few_with_new_url = convert_fetched_lyrics_to_df(
    once_again_with_better_url,
    better_url_refetched_lyrics[
        better_url_refetched_lyrics.lyrics.isna()
    ]
)

_put_back_in = pd.concat([
    lol_the_had_to_refetch_a_few_with_new_url,
    better_url_refetched_lyrics.dropna()
])

save_lyrics(
    _put_back_in,
    'Refetched with more likely URL (good to go now!)'
)

,song_id,lyrics,urls,likely_correct_url
11,Drop It Like It's HotSnoop Dogg Featuring Phar...,TranslationsPortuguêsDrop It Like It’s Hot Lyr...,NaN,https://genius.com/Snoop-dogg-drop-it-like-its...
17,Flashdance...What A FeelingIrene Cara,Flashdance... What a Feeling Lyrics[Intro]\nFi...,NaN,https://genius.com/Irene-cara-flashdance-what-...
27,I Love ItIcona Pop Featuring Charli XCX,TranslationsPortuguêsI Love It Lyrics[Chorus: ...,NaN,https://genius.com/Icona-pop-i-love-it-lyrics
33,Let Me Blow Ya MindEve Featuring Gwen Stefani,"Let Me Blow Ya Mind Lyrics[Intro: Eve]\nUh, uh...",NaN,https://genius.com/Eve-let-me-blow-ya-mind-lyrics
40,Moves Like JaggerMaroon 5 Featuring Christina ...,Moves Like Jagger Lyrics[Intro: Adam Levine]\n...,NaN,https://genius.com/Maroon-5-moves-like-jagger-...
...,...,...,...,...
64,We Are Youngfun. Featuring Janelle Monae,We Are Young Lyrics[Verse 1: Nate Ruess]\nGive...,"[""https://genius.com/Adoseofbuckley-the-ten-wo...",https://genius.com/Fun-we-are-young-lyrics
65,We Found LoveRihanna Featuring Calvin Harris,TranslationsPortuguêsEnglishFrançaisWe Found L...,"[""https://genius.com/Npr-the-200-greatest-song...",https://genius.com/Rihanna-we-found-love-lyrics
66,We R Who We RKe$ha,TranslationsPortuguêsWe R Who We R Lyrics[Vers...,"[""https://genius.com/Adoseofbuckley-adoseofbuc...",https://genius.com/Kesha-we-r-who-we-r-lyrics
67,Young ForeverJay-Z + Mr. Hudson,Young Forever Lyrics[Intro: Mr. Hudson]\nLet's...,"[""https://genius.com/Genius-deutschland-die-li...",https://genius.com/Jay-z-young-forever-lyrics


## ✅ 3. 400 songs that failed to fetch (for many reasons) originally, which I refetched refetched.  🎉 Hurray: of these refetches that got *urls but not lyrics*, all that failed were *instrumentals* -- SIMPLE. I saved a seperate file for those on disk.

In [17]:
big_refetch_of_misc = pd.read_csv(f'{LYRIC_FOLDER_PATH}/{"TODO this is the result of refetching 400 non-lyriced songs (some are instrumentals turns out)"}.csv')
big_refetch_of_misc


,song_id,lyrics,urls
0,(Now and Then There's) A Fool Such As IElvis P...,NaN,NaN
1,(Sittin' On) The Dock Of The BayOtis Redding,(Sittin’ On) The Dock of the Bay Lyrics[Verse ...,"[""https://genius.com/Otis-redding-sittin-on-th..."
2,(You Gotta) Fight For Your Right (To Party!)Be...,Fight for Your Right Lyrics[Intro]\nYeah\nKick...,"[""https://genius.com/Beastie-boys-fight-for-yo..."
3,(You're the) Devil In DisguiseElvis Presley Wi...,NaN,NaN
4,...Baby One More TimeBritney Spears,TranslationsItaliano...Baby One More Time Lyri...,"[""https://genius.com/Britney-spears-baby-one-m..."
...,...,...,...
418,"You're The First, The Last, My EverythingBarry...",NaN,NaN
419,Youngblood5 Seconds Of Summer,TranslationsEspañolPortuguêsYoungblood Lyrics[...,"[""https://genius.com/5-seconds-of-summer-young..."
420,Your LoveThe Outfield,NaN,NaN
421,YummyJustin Bieber,NaN,NaN


### 1. FIRST, let's try re-fetching the many songs that failed to even get URLS


(NOTE: the ones that have URLS but not lyrics turn out to all be *instrumentals*)

### 1a. do the refetch using *just* song name (to see if it succeeds)

In [18]:
_fetch_totally_failed = big_refetch_of_misc[
    big_refetch_of_misc.urls.isna() &
    big_refetch_of_misc.lyrics.isna()
]

In [19]:
_fetch_totally_failed.merge(
    on='song_id',
    right=songs_df,
    how='left'
)

,song_id,lyrics,urls,chart_position,chart_debut,song,performer,artist_array,generic_genre
0,(Now and Then There's) A Fool Such As IElvis P...,NaN,NaN,2,1959-03-21,(Now and Then There's) A Fool Such As I,Elvis Presley With The Jordanaires,['Elvis Presley With The Jordanaires'],pop
1,(You're the) Devil In DisguiseElvis Presley Wi...,NaN,NaN,3,1963-06-29,(You're the) Devil In Disguise,Elvis Presley With The Jordanaires,['Elvis Presley With The Jordanaires'],pop
2,A Big Hunk O' LoveElvis Presley With The Jorda...,NaN,NaN,1,1959-07-04,A Big Hunk O' Love,Elvis Presley With The Jordanaires,['Elvis Presley With The Jordanaires'],pop
3,A Horse With No NameAmerica,NaN,NaN,1,1972-02-19,A Horse With No Name,America,['America'],hip hop
4,A Million To OneJimmy Charles and The Revelletts,NaN,NaN,5,1960-08-20,A Million To One,Jimmy Charles and The Revelletts,['Jimmy Charles and The Revelletts'],folk
...,...,...,...,...,...,...,...,...,...
190,You Turn Me On (Turn On Song)Ian Whitcomb And ...,NaN,NaN,8,1965-05-22,You Turn Me On (Turn On Song),Ian Whitcomb And Bluesville,['Ian Whitcomb And Bluesville'],uncategorized
191,"You're The First, The Last, My EverythingBarry...",NaN,NaN,2,1974-11-02,"You're The First, The Last, My Everything",Barry White,['Barry White'],pop
192,Your LoveThe Outfield,NaN,NaN,6,1986-02-15,Your Love,The Outfield,['The Outfield'],NaN
193,YummyJustin Bieber,NaN,NaN,2,2020-01-18,Yummy,Justin Bieber,['Justin Bieber'],country


In [29]:
def _search_song_chose_first_then_get_lyrics(song, performer):
    try: 
        print(f'{performer} {song}')
        print('CLEANED: ', clean_it(f'{performer} {song}'))
#         songs = genius.search_songs(clean_it(f'{performer} {song}'))
        # TRICK: just search the song name only
        songs = genius.search_songs(clean_it(f'{song}'))
        urls = [song['result']['url'] for song in songs['hits']]
        print(urls)
        return [genius.lyrics(song_url=urls[0]), urls]
    except:
        print(f'OOPS: {performer} {song}')

In [30]:
def TROUBLE_SHOOTTING_fetch_lyrics_and_urls(songs_subset_df):
    return songs_subset_df.apply(
        lambda row: _search_song_chose_first_then_get_lyrics(song=row['song'], performer=row['performer']),
        axis=1
    )

In [35]:
_just_song_name = TROUBLE_SHOOTTING_fetch_lyrics_and_urls(
    _fetch_totally_failed.merge(
        on='song_id',
        right=songs_df,
        how='left'
    )
)

Elvis Presley With The Jordanaires (Now and Then There's) A Fool Such As I
CLEANED:  Elvis Presley With The Jordanaires Now and Then Theres A Fool Such As I
['https://genius.com/Hank-snow-now-and-then-theres-a-fool-such-as-i-lyrics', 'https://genius.com/Elvis-presley-now-and-then-theres-a-fool-such-as-i-lyrics', 'https://genius.com/Rodney-crowell-now-and-then-theres-a-fool-such-as-i-lyrics', 'https://genius.com/Willie-nelson-and-hank-snow-now-and-then-theres-a-fool-such-as-i-lyrics', 'https://genius.com/Steve-goodman-now-and-then-theres-a-fool-such-as-i-lyrics', 'https://genius.com/Jim-reeves-now-and-then-theres-a-fool-such-as-i-lyrics', 'https://genius.com/Raul-malo-now-and-then-theres-a-fool-such-as-i-lyrics', 'https://genius.com/Doris-day-now-and-then-theres-a-fool-such-as-i-lyrics', 'https://genius.com/Bobby-helms-now-and-then-theres-a-fool-such-as-i-lyrics', 'https://genius.com/Boots-randolph-now-and-then-theres-a-fool-such-as-i-lyrics']
Elvis Presley With The Jordanaires (You're 

Jorgen Ingmann & His Guitar Apache
CLEANED:  Jorgen Ingmann & His Guitar Apache
['https://genius.com/Apache-207-roller-lyrics', 'https://genius.com/Apache-207-wieso-tust-du-dir-das-an-lyrics', 'https://genius.com/Apache-207-fame-lyrics', 'https://genius.com/Sido-and-apache-207-2002-lyrics', 'https://genius.com/Apache-207-200-km-h-lyrics', 'https://genius.com/Apache-207-matrix-lyrics', 'https://genius.com/Sugarhill-gang-apache-jump-on-it-lyrics', 'https://genius.com/Apache-207-angst-lyrics', 'https://genius.com/Apache-207-unterwegs-lyrics', 'https://genius.com/Apache-207-blaulich-lyrics']
Elvis Presley With The Jordanaires Are You Lonesome To-night?
CLEANED:  Elvis Presley With The Jordanaires Are You Lonesome To-night?
['https://genius.com/Elvis-presley-are-you-lonesome-tonight-lyrics', 'https://genius.com/Chris-botti-are-you-lonesome-tonight-lyrics', 'https://genius.com/Donny-osmond-are-you-lonesome-tonight-lyrics', 'https://genius.com/Brian-hyland-are-you-lonesome-tonight-lyrics', 'h

Dan Hill (Duet With Vonda Shepard) Can't We Try
CLEANED:  Dan Hill Duet With Vonda Shepard Cant We Try
['https://genius.com/Teddy-pendergrass-cant-we-try-lyrics', 'https://genius.com/Rockell-cant-we-try-lyrics', 'https://genius.com/Larry-santos-cant-we-try-again-lyrics', 'https://genius.com/Thelma-houston-cant-we-try-lyrics', 'https://genius.com/Collage-cant-we-try-lyrics', 'https://genius.com/Dan-hill-cant-we-try-single-version-lyrics', 'https://genius.com/Clem-clempson-cant-we-try-again-lyrics', 'https://genius.com/Bruno-mars-too-good-to-say-goodbye-lyrics', 'https://genius.com/Ella-vos-you-dont-know-about-me-lyrics', 'https://genius.com/Mary-j-blige-marys-joint-lyrics']
Elton John Candle In The Wind 1997/Something About The Way You Look Tonight
CLEANED:  Elton John Candle In The Wind 1997Something About The Way You Look Tonight
[]
OOPS: Elton John Candle In The Wind 1997/Something About The Way You Look Tonight
Gary Numan Cars
CLEANED:  Gary Numan Cars
['https://genius.com/Snow-patr

Lonnie Donegan And His Skiffle Group Does Your Chewing Gum Lose It's Flavor (On The Bedpost Over Night)
CLEANED:  Lonnie Donegan And His Skiffle Group Does Your Chewing Gum Lose Its Flavor On The Bedpost Over Night
['https://genius.com/Shane-black-kiss-kiss-bang-bang-script-annotated']
Electric Light Orchestra Don't Bring Me Down
CLEANED:  Electric Light Orchestra Dont Bring Me Down
['https://genius.com/Electric-light-orchestra-dont-bring-me-down-lyrics', 'https://genius.com/Sia-dont-bring-me-down-lyrics', 'https://genius.com/Two-feet-dont-bring-me-down-lyrics', 'https://genius.com/Prep-dont-bring-me-down-lyrics', 'https://genius.com/The-animals-dont-bring-me-down-lyrics', 'https://genius.com/Black-eyed-peas-dont-bring-me-down-lyrics', 'https://genius.com/David-bowie-dont-bring-me-down-lyrics', 'https://genius.com/The-pretty-things-dont-bring-me-down-lyrics', 'https://genius.com/Tom-petty-and-the-heartbreakers-dont-bring-me-down-lyrics', 'https://genius.com/Hippo-campus-dont-bring-me-d

Backstreet Boys Everybody [Backstreet's Back]
CLEANED:  Backstreet Boys Everybody Backstreets Back
['https://genius.com/Backstreet-boys-everybody-backstreets-back-lyrics', 'https://genius.com/Backstreet-boys-everybody-backstreets-back-radio-edit-lyrics', 'https://genius.com/Backstreet-boys-everybody-backstreets-back-extended-version-lyrics', 'https://genius.com/Operacion-triunfo-2020-everybody-backstreets-back-lyrics', 'https://genius.com/Millennium-alert-everybody-backstreets-back-lyrics', 'https://genius.com/Backstreet-boys-everybody-backstreets-back-alternative-extended-version-lyrics', 'https://genius.com/Backstreet-boys-everybody-backstreets-back-complete-version-lyrics', 'https://genius.com/Backstreet-boys-everybody-backstreets-back-funked-up-mix-lyrics', 'https://genius.com/Backstreet-boys-everybody-backstreets-back-max-and-macario-club-mix-lyrics', 'https://genius.com/Backstreet-boys-everybody-backstreets-back-mattys-remix-lyrics']
James Taylor Fire And Rain
CLEANED:  James Tay

...

['https://genius.com/Lady-gaga-and-ariana-grande-rain-on-me-lyrics', 'https://genius.com/Joji-rain-on-me-lyrics', 'https://genius.com/Bryson-tiller-rain-on-me-intro-lyrics', 'https://genius.com/Ashanti-rain-on-me-lyrics', 'https://genius.com/Travis-why-does-it-always-rain-on-me-lyrics', 'https://genius.com/Lady-gaga-ariana-grande-and-arca-rain-on-me-arca-remix-lyrics', 'https://genius.com/Genius-traducciones-al-espanol-lady-gaga-and-ariana-grande-rain-on-me-traduccion-al-espanol-lyrics', 'https://genius.com/Ajj-hate-rain-on-me-lyrics', 'https://genius.com/Trophy-eyes-rain-on-me-lyrics', 'https://genius.com/Kidz-bop-kids-rain-on-me-2020-lyrics']
Brook Benton Rainy Night In Georgia/Rubberneckin'
CLEANED:  Brook Benton Rainy Night In GeorgiaRubberneckin
[]
OOPS: Brook Benton Rainy Night In Georgia/Rubberneckin'
Duane Eddy His Twangy Guitar And The Rebels Rebel-'rouser
CLEANED:  Duane Eddy His Twangy Guitar And The Rebels Rebel-rouser
['https://genius.com/Duane-eddy-rebel-rouser-lyrics', '

OOPS: WALK THE MOON Shut Up And Dance
Deep Purple Smoke On The Water
CLEANED:  Deep Purple Smoke On The Water
['https://genius.com/Deep-purple-smoke-on-the-water-lyrics', 'https://genius.com/Red-foley-smoke-on-the-water-lyrics', 'https://genius.com/Rock-aid-armenia-smoke-on-the-water-lyrics', 'https://genius.com/Santana-smoke-on-the-water-lyrics', 'https://genius.com/Black-sabbath-smoke-on-the-water-lyrics', 'https://genius.com/Pesochnye-lyudi-smoke-on-the-water-lyrics', 'https://genius.com/Pat-boone-smoke-on-the-water-lyrics', 'https://genius.com/Ian-gillan-smoke-on-the-water-lyrics', 'https://genius.com/Mr-big-smoke-on-the-water-lyrics', 'https://genius.com/Six-feet-under-smoke-on-the-water-lyrics']
Sade Smooth Operator
CLEANED:  Sade Smooth Operator
['https://genius.com/Sade-smooth-operator-lyrics', 'https://genius.com/Yung-bleu-smooth-operator-lyrics', 'https://genius.com/Kizaru-smooth-operator-lyrics', 'https://genius.com/Big-daddy-kane-smooth-operator-lyrics', 'https://genius.com

['https://genius.com/Corey-hart-sunglasses-at-night-lyrics', 'https://genius.com/Skepta-sunglasses-at-night-lyrics', 'https://genius.com/The-megas-sunglasses-at-night-lyrics', 'https://genius.com/Javier-martinez-maya-i-wear-my-sunglasses-at-night-lyrics', 'https://genius.com/Voodoo-and-serano-sunglasses-at-night-lyrics', 'https://genius.com/Alok-and-hollaphonic-sunglasses-at-night-lyrics', 'https://genius.com/Merk-la-familia-sunglasses-at-night-lyrics', 'https://genius.com/Eastblock-bitches-x-ostblockschlampen-sunglasses-at-night-lyrics', 'https://genius.com/Corey-hart-sunglasses-at-night-jean-jacket-mix-lyrics', 'https://genius.com/Poppy-sunglasses-at-night-lyrics']
Carpenters Superstar/Bless The Beasts And Children
CLEANED:  Carpenters SuperstarBless The Beasts And Children
[]
OOPS: Carpenters Superstar/Bless The Beasts And Children
Ann Wilson & Robin Zander Surrender To Me (From "Tequila Sunrise")
CLEANED:  Ann Wilson & Robin Zander Surrender To Me From Tequila Sunrise
['https://gen

Rod Stewart With Ronald Isley This Old Heart Of Mine
CLEANED:  Rod Stewart With Ronald Isley This Old Heart Of Mine
['https://genius.com/The-isley-brothers-this-old-heart-of-mine-is-weak-for-you-lyrics', 'https://genius.com/Rod-stewart-this-old-heart-of-mine-lyrics', 'https://genius.com/The-temptations-this-old-heart-of-mine-is-weak-for-you-lyrics', 'https://genius.com/The-zombies-this-old-heart-of-mine-live-lyrics', 'https://genius.com/Wild-cherry-this-old-heart-of-mine-is-weak-for-you-lyrics', 'https://genius.com/The-supremes-this-old-heart-of-mine-is-weak-for-you-lyrics', 'https://genius.com/The-isley-brothers-this-old-heart-of-mine-is-weak-for-you-alternate-mix-lyrics', 'https://genius.com/A-loss-for-words-this-old-heart-of-mine-lyrics', 'https://genius.com/Boyzone-this-old-heart-of-mine-lyrics']
Commodores Three Times A Lady
CLEANED:  Commodores Three Times A Lady
['https://genius.com/Commodores-three-times-a-lady-lyrics', 'https://genius.com/Kenny-rogers-three-times-a-lady-lyrics

John Lennon With The Plastic Ono Nuclear Band Whatever Gets You Thru The Night
CLEANED:  John Lennon With The Plastic Ono Nuclear Band Whatever Gets You Thru The Night
['https://genius.com/John-lennon-whatever-gets-you-thru-the-night-lyrics', 'https://genius.com/Los-lonely-boys-whatever-gets-you-thru-the-night-lyrics', 'https://genius.com/Bryan-ferry-whatever-gets-you-thru-the-night-lyrics', 'https://genius.com/John-lennon-whatever-gets-you-thru-the-night-live-version-lyrics', 'https://genius.com/The-lonely-boys-whatever-gets-you-thru-the-night-lyrics', 'https://genius.com/Tre-wavvy-vengeance-lyrics', 'https://genius.com/Jboat-the-rapper-late-night-bike-rides-lyrics', 'https://genius.com/Loick-essien-whatever-you-like-lyrics', 'https://genius.com/Samantha-fox-what-you-see-is-what-you-get-lyrics', 'https://genius.com/Jr-rhodes-thru-the-night-feat-topboypoison-lyrics']
Jimmy Harnen With Synch Where Are You Now?
CLEANED:  Jimmy Harnen With Synch Where Are You Now?
['https://genius.com/Lad

In [75]:
_fetched_just_name_df = convert_fetched_lyrics_to_df(_just_song_name, _fetch_totally_failed)
_fetched_just_name_df = _fetched_just_name_df.merge(on="song_id", right=songs_df, how='left')

### 1b. Of the *new urls* succesfully fetched (ignoring those that *still* failed), I want to check the Levenshtein distance to be safe, just-song-name search could easily yield bogus urls

In [87]:
from Levenshtein import distance
import json

def return_likely_correct_url_or_else_false(row):
    urls = row['urls'] and json.loads(row['urls'])
    if not urls:
        return
#     print(row['performer'], row['song'])
    for url in urls:
#         print(url)
        if url.endswith('-lyrics'): 
            dist = get_l_dist(row['performer'], row['song'], url)
            contains_artist_name = row['performer'].split()[0] in url
            if dist < 13 or contains_artist_name:
                return url
    return False

a ='Lil Wayne Featuring Cory Gunz'  
s = '6 Foot 7 Foot'
u = 'https://genius.com/Lil-wayne-6-foot-7-foot-lyrics'
def get_l_dist(performer, song, url):
    format_song = f'{performer.split()[0]} {song} lyrics'
    format_url = url.replace('https://genius.com/', '').replace('-', ' ')
    return distance(
        format_song,
        format_url
    )



In [91]:
_fetched_just_name_df['likely_correct_url'] = _fetched_just_name_df.apply(return_likely_correct_url_or_else_false, axis=1)
_fetched_just_name_df

save_lyrics(_fetched_just_name_df, 'TEMP just hold onto the song-name-only refetches from the 400 misc that did not fetch originally')


,song_id,lyrics,urls,chart_position,chart_debut,song,performer,artist_array,generic_genre,likely_correct_url
0,(Now and Then There's) A Fool Such As IElvis P...,"(Now And Then, There’s) A Fool Such As I Lyric...","[""https://genius.com/Hank-snow-now-and-then-th...",2,1959-03-21,(Now and Then There's) A Fool Such As I,Elvis Presley With The Jordanaires,['Elvis Presley With The Jordanaires'],pop,https://genius.com/Elvis-presley-now-and-then-...
1,(You're the) Devil In DisguiseElvis Presley Wi...,(You’re The) Devil In Disguise Lyrics[Pre-Chor...,"[""https://genius.com/Elvis-presley-youre-the-d...",3,1963-06-29,(You're the) Devil In Disguise,Elvis Presley With The Jordanaires,['Elvis Presley With The Jordanaires'],pop,https://genius.com/Elvis-presley-youre-the-dev...
2,A Big Hunk O' LoveElvis Presley With The Jorda...,"A Big Hunk O’ Love Lyrics[Intro]\nHey baby, I ...","[""https://genius.com/Elvis-presley-a-big-hunk-...",1,1959-07-04,A Big Hunk O' Love,Elvis Presley With The Jordanaires,['Elvis Presley With The Jordanaires'],pop,https://genius.com/Elvis-presley-a-big-hunk-o-...
3,A Horse With No NameAmerica,A Horse With No Name Lyrics[Verse 1]\nOn the f...,"[""https://genius.com/America-a-horse-with-no-n...",1,1972-02-19,A Horse With No Name,America,['America'],hip hop,https://genius.com/America-a-horse-with-no-nam...
4,A Million To OneJimmy Charles and The Revelletts,"A Million to One Lyrics[Verse 1]\nBaby, now th...","[""https://genius.com/Kiss-a-million-to-one-lyr...",5,1960-08-20,A Million To One,Jimmy Charles and The Revelletts,['Jimmy Charles and The Revelletts'],folk,https://genius.com/Kiss-a-million-to-one-lyrics
...,...,...,...,...,...,...,...,...,...,...
190,You Turn Me On (Turn On Song)Ian Whitcomb And ...,Someone to Talk About Lyrics[Verse 1: Grace Mi...,"[""https://genius.com/Phantoms-someone-to-talk-...",8,1965-05-22,You Turn Me On (Turn On Song),Ian Whitcomb And Bluesville,['Ian Whitcomb And Bluesville'],uncategorized,False
191,"You're The First, The Last, My EverythingBarry...","You’re the First, the Last, My Everything Lyri...","[""https://genius.com/Barry-white-youre-the-fir...",2,1974-11-02,"You're The First, The Last, My Everything",Barry White,['Barry White'],pop,https://genius.com/Barry-white-youre-the-first...
192,Your LoveThe Outfield,TranslationsPortuguêsHow Deep Is Your Love Lyr...,"[""https://genius.com/Calvin-harris-and-discipl...",6,1986-02-15,Your Love,The Outfield,['The Outfield'],NaN,https://genius.com/Drake-find-your-love-lyrics
193,YummyJustin Bieber,TranslationsTürkçeSvenskaEspañolРусскийPortugu...,"[""https://genius.com/Justin-bieber-yummy-lyric...",2,2020-01-18,Yummy,Justin Bieber,['Justin Bieber'],country,https://genius.com/Justin-bieber-yummy-lyrics


### 1c. Determine when the `likely_correct_url` is correct, & *hand code* when it fails

In [191]:
just_more_likely_urls = _fetched_just_name_df.dropna(subset=['likely_correct_url'])

In [206]:
_where_urls_differ = just_more_likely_urls[
    just_more_likely_urls.apply(lambda row: json.loads(row['urls'])[0] != row['likely_correct_url'], axis=1) &
    just_more_likely_urls.likely_correct_url.apply(lambda v: v is not False)
]
_where_urls_differ

,song_id,lyrics,urls,chart_position,chart_debut,song,performer,artist_array,generic_genre,likely_correct_url
0,(Now and Then There's) A Fool Such As IElvis P...,"(Now And Then, There’s) A Fool Such As I Lyric...","[""https://genius.com/Hank-snow-now-and-then-th...",2,1959-03-21,(Now and Then There's) A Fool Such As I,Elvis Presley With The Jordanaires,['Elvis Presley With The Jordanaires'],pop,https://genius.com/Elvis-presley-now-and-then-...
19,"Breaking FreeZac Efron, Andrew Seeley & Vaness...",TranslationsFrançaisBreaking Free Lyrics[Intro...,"[""https://genius.com/High-school-musical-cast-...",4,2006-02-04,Breaking Free,"Zac Efron, Andrew Seeley & Vanessa Anne Hudgens","['Zac Efron', 'Andrew Seeley', 'Vanessa Anne H...",rock,https://genius.com/Skillet-breaking-free-lyrics
21,Can't Fight This FeelingREO Speedwagon,"Can’t Fight This Feeling Lyrics[Verse 1]\nOh, ...","[""https://genius.com/Reo-speedwagon-cant-fight...",1,1985-01-19,Can't Fight This Feeling,REO Speedwagon,['REO Speedwagon'],pop,https://genius.com/Smokie-cant-fight-this-feel...
24,Can't We TryDan Hill (Duet With Vonda Shepard),Can’t We Try Lyrics[Verse 1]\nCome lay your he...,"[""https://genius.com/Teddy-pendergrass-cant-we...",6,1987-06-06,Can't We Try,Dan Hill (Duet With Vonda Shepard),['Dan Hill (Duet With Vonda Shepard)'],hip hop,https://genius.com/Rockell-cant-we-try-lyrics
26,CarsGary Numan,TranslationsEspañolChasing Cars Lyrics[Verse 1...,"[""https://genius.com/Snow-patrol-chasing-cars-...",9,1980-02-16,Cars,Gary Numan,['Gary Numan'],uncategorized,https://genius.com/The-cars-drive-lyrics
45,DreamsFleetwood Mac,TranslationsTürkçeРусскийRomânăPortuguêsItalia...,"[""https://genius.com/Juice-wrld-lucid-dreams-l...",1,1977-04-16,Dreams,Fleetwood Mac,['Fleetwood Mac'],uncategorized,https://genius.com/Fleetwood-mac-dreams-lyrics
52,"For What It's Worth (Stop, Hey What's That Sou...",For What It’s Worth Lyrics[Verse 1]\nThere's s...,"[""https://genius.com/Buffalo-springfield-for-w...",7,1967-01-28,"For What It's Worth (Stop, Hey What's That Sound)",The Buffalo Springfield,['The Buffalo Springfield'],electro,https://genius.com/The-muppets-for-what-its-wo...
53,FrankensteinEdgar Winter Group,TranslationsEspañolFrankenstein Lyrics[Part I]...,"[""https://genius.com/Swedish-house-mafia-and-a...",1,1973-03-10,Frankenstein,Edgar Winter Group,['Edgar Winter Group'],pop,https://genius.com/Babytron-frankenstein-lyrics
54,FriendsJody Watley With Eric B. & Rakim,TranslationsРусскийTürkçeFRIENDS Lyrics[Intro]...,"[""https://genius.com/Marshmello-and-anne-marie...",9,1989-06-17,Friends,Jody Watley With Eric B. & Rakim,"['Jody Watley With Eric B.', 'Rakim']",uncategorized,https://genius.com/The-carters-friends-lyrics
56,GhostbustersRay Parker Jr.,Ghost Busters Lyrics[Intro: Nicole Zymek]\nHon...,"[""https://genius.com/Trippie-redd-and-xxxtenta...",1,1984-06-16,Ghostbusters,Ray Parker Jr.,['Ray Parker Jr.'],rock,https://genius.com/Ray-parker-jr-ghostbusters-...


#### Determine by hand which URLS are not right in `likely_correct_url`, list them here if so

In [231]:
# assume likely_correct_url is right UNLESS it's listed below

EVEN_MORE_INSTRUMENTALS_FOUND = [
    'FrankensteinEdgar Winter Group',
    'Watermelon ManMongo Santamaria Band',
    'The Good, The Bad And The UglyHugo Montenegro, His Orchestra And Chorus',
]

LYRICS_CORRECTED_BY_HAND = {
    'Breaking FreeZac Efron, Andrew Seeley & Vanessa Anne Hudgens': 'https://genius.com/High-school-musical-cast-breaking-free-lyrics',
    "Can't Fight This FeelingREO Speedwagon": 'https://genius.com/Reo-speedwagon-cant-fight-this-feeling-lyrics',
    "Can't We TryDan Hill (Duet With Vonda Shepard)": 'https://genius.com/Dan-hill-cant-we-try-single-version-lyrics',
    'CarsGary Numan	': 'https://genius.com/Gary-numan-cars-lyrics',
    "For What It's Worth (Stop, Hey What's That Sound)The Buffalo Springfield": 'https://genius.com/Buffalo-springfield-for-what-its-worth-lyrics',
    "FriendsJody Watley With Eric B. & Rakim": 'https://genius.com/Jody-watley-friends-lyrics',
    "I'm Gonna Love You Just A Little More BabyBarry White": 'https://genius.com/Barry-white-im-gonna-love-you-just-a-little-more-baby-lyrics',
    "Jungle FeverThe Chakachas": 'https://genius.com/Chakachas-jungle-fever-lyrics',
    "Keep On Loving YouREO Speedwagon": 'https://genius.com/Reo-speedwagon-keep-on-loving-you-lyrics',
    "MickeyToni Basil": 'https://genius.com/Toni-basil-mickey-lyrics',
    "PatriciaPerez Prado And His Orchestra": 'https://genius.com/Perez-prado-patricia-lyrics',
    "SurrenderElvis Presley With The Jordanaires": 'https://genius.com/Elvis-presley-surrender-lyrics',
    "Wonderland By NightBert Kaempfert And His Orchestra": 'https://genius.com/Bert-kaempfert-wonderland-by-night-lyrics',
    "Your LoveThe Outfield": 'https://genius.com/The-outfield-your-love-lyrics',
    
}

In [232]:
with pd.option_context('display.max_colwidth', None,
                       'display.max_columns', None,
                       'display.max_rows', None):
    display(_where_urls_differ[['song_id', 'lyrics', 'urls', 'likely_correct_url']].reset_index())

index  \
0       0   
1      19   
2      21   
3      24   
4      26   
5      45   
6      52   
7      53   
8      54   
9      56   
10     58   
11     69   
12     84   
13     88   
14     95   
15     97   
16     99   
17    110   
18    121   
19    123   
20    126   
21    130   
22    161   
23    162   
24    164   
25    165   
26    169   
27    170   
28    171   
29    178   
30    181   
31    186   
32    188   
33    192   

                                                                                                   song_id  \
0                                (Now and Then There's) A Fool Such As IElvis Presley With The Jordanaires   
1                                             Breaking FreeZac Efron, Andrew Seeley & Vanessa Anne Hudgens   
2                                                                   Can't Fight This FeelingREO Speedwagon   
3                                                           Can't We TryDan Hill (Duet With Vonda Shepard)   
4                                                                                           CarsGary Numan   
5                                                                                      DreamsFleetwood Mac   
6                                 For What It's Worth (Stop, Hey What's That Sound)The Buffalo Springfield   
7                                                                           FrankensteinEdgar Winter Group   
8                                                                  FriendsJody Watley With Eric B. & Rakim   
9                                                                               GhostbustersRay Parker Jr.   
10                                                       Good Luck CharmElvis Presley With The Jordanaires   
11                                                                        I Can See Clearly NowJohnny Nash   
12                                                   I'm Gonna Love You Just A Little More BabyBarry White   
13                                                                       If I Can't Have YouYvonne Elliman   
14                                                                               Jungle FeverThe Chakachas   
15                                                                        Keep On Loving YouREO Speedwagon   
16                                                                                   Lady MarmaladeLabelle   
17                                                                                        MickeyToni Basil   
18                                  Not TonightLil' Kim Feat. Da Brat, Left Eye, Missy Elliott & Angie Mar   
19                                                                             Off The WallMichael Jackson   
20                                                                   PatriciaPerez Prado And His Orchestra   
21                                                                                    PoisonBell Biv DeVoe   
22                                                             SurrenderElvis Presley With The Jordanaires   
23                                                                    Tall PaulAnnette With The Afterbeats   
24                                 The Good, The Bad And The UglyHugo Montenegro, His Orchestra And Chorus   
25                                                                     The Most Beautiful GirlCharlie Rich   
26  This Is For The Lover In YouBabyface Featuring LL Cool J, Howard Hewett, Jody Watley & Jeffrey Daniels   
27                                                                                       This LoveMaroon 5   
28                                                     This Old Heart Of MineRod Stewart With Ronald Isley   
29                                                                     Watermelon ManMongo Santamaria Band   
30                                                        What The World Needs Now Is LoveJackie DeShannon   
31                          

#### So, using our list of URLs that need to be refetched, we can refetch those

In [236]:
_temp = _where_urls_differ.copy()[
    _where_urls_differ.song_id.apply(lambda i: i not in EVEN_MORE_INSTRUMENTALS_FOUND)
]

def _update_urls(row):
    if row['song_id'] in LYRICS_CORRECTED_BY_HAND:
        return LYRICS_CORRECTED_BY_HAND[row['song_id']] 
    return row['likely_correct_url']


_temp['likely_correct_url'] = _temp.apply(_update_urls, axis=1)
_temp

,song_id,lyrics,urls,chart_position,chart_debut,song,performer,artist_array,generic_genre,likely_correct_url
0,(Now and Then There's) A Fool Such As IElvis P...,"(Now And Then, There’s) A Fool Such As I Lyric...","[""https://genius.com/Hank-snow-now-and-then-th...",2,1959-03-21,(Now and Then There's) A Fool Such As I,Elvis Presley With The Jordanaires,['Elvis Presley With The Jordanaires'],pop,https://genius.com/Elvis-presley-now-and-then-...
19,"Breaking FreeZac Efron, Andrew Seeley & Vaness...",TranslationsFrançaisBreaking Free Lyrics[Intro...,"[""https://genius.com/High-school-musical-cast-...",4,2006-02-04,Breaking Free,"Zac Efron, Andrew Seeley & Vanessa Anne Hudgens","['Zac Efron', 'Andrew Seeley', 'Vanessa Anne H...",rock,https://genius.com/High-school-musical-cast-br...
21,Can't Fight This FeelingREO Speedwagon,"Can’t Fight This Feeling Lyrics[Verse 1]\nOh, ...","[""https://genius.com/Reo-speedwagon-cant-fight...",1,1985-01-19,Can't Fight This Feeling,REO Speedwagon,['REO Speedwagon'],pop,https://genius.com/Reo-speedwagon-cant-fight-t...
24,Can't We TryDan Hill (Duet With Vonda Shepard),Can’t We Try Lyrics[Verse 1]\nCome lay your he...,"[""https://genius.com/Teddy-pendergrass-cant-we...",6,1987-06-06,Can't We Try,Dan Hill (Duet With Vonda Shepard),['Dan Hill (Duet With Vonda Shepard)'],hip hop,https://genius.com/Dan-hill-cant-we-try-single...
26,CarsGary Numan,TranslationsEspañolChasing Cars Lyrics[Verse 1...,"[""https://genius.com/Snow-patrol-chasing-cars-...",9,1980-02-16,Cars,Gary Numan,['Gary Numan'],uncategorized,https://genius.com/The-cars-drive-lyrics
45,DreamsFleetwood Mac,TranslationsTürkçeРусскийRomânăPortuguêsItalia...,"[""https://genius.com/Juice-wrld-lucid-dreams-l...",1,1977-04-16,Dreams,Fleetwood Mac,['Fleetwood Mac'],uncategorized,https://genius.com/Fleetwood-mac-dreams-lyrics
52,"For What It's Worth (Stop, Hey What's That Sou...",For What It’s Worth Lyrics[Verse 1]\nThere's s...,"[""https://genius.com/Buffalo-springfield-for-w...",7,1967-01-28,"For What It's Worth (Stop, Hey What's That Sound)",The Buffalo Springfield,['The Buffalo Springfield'],electro,https://genius.com/Buffalo-springfield-for-wha...
54,FriendsJody Watley With Eric B. & Rakim,TranslationsРусскийTürkçeFRIENDS Lyrics[Intro]...,"[""https://genius.com/Marshmello-and-anne-marie...",9,1989-06-17,Friends,Jody Watley With Eric B. & Rakim,"['Jody Watley With Eric B.', 'Rakim']",uncategorized,https://genius.com/Jody-watley-friends-lyrics
56,GhostbustersRay Parker Jr.,Ghost Busters Lyrics[Intro: Nicole Zymek]\nHon...,"[""https://genius.com/Trippie-redd-and-xxxtenta...",1,1984-06-16,Ghostbusters,Ray Parker Jr.,['Ray Parker Jr.'],rock,https://genius.com/Ray-parker-jr-ghostbusters-...
58,Good Luck CharmElvis Presley With The Jordanaires,"Good Luck Charm Lyrics[Intro: B. Casey]\nJ.E.,...","[""https://genius.com/Jagged-edge-good-luck-cha...",1,1962-03-17,Good Luck Charm,Elvis Presley With The Jordanaires,['Elvis Presley With The Jordanaires'],pop,https://genius.com/Elvis-presley-good-luck-cha...


In [234]:
# NA: refetch just these

def _get_song(url):
    try: 
        print(url, '...')
        return [genius.lyrics(song_url=url), [url]]
    except:
        print(f'OOPS: {url}')
        
def refetch_lyrics_and_urls(songs_subset_df):
    return songs_subset_df.apply(
        lambda row: _get_song(row['likely_correct_url']),
        axis=1
    )
res = refetch_lyrics_and_urls(_temp)

res

https://genius.com/Elvis-presley-now-and-then-theres-a-fool-such-as-i-lyrics ...
https://genius.com/High-school-musical-cast-breaking-free-lyrics ...
https://genius.com/Reo-speedwagon-cant-fight-this-feeling-lyrics ...
https://genius.com/Dan-hill-cant-we-try-single-version-lyrics ...
https://genius.com/The-cars-drive-lyrics ...
https://genius.com/Fleetwood-mac-dreams-lyrics ...
https://genius.com/Buffalo-springfield-for-what-its-worth-lyrics ...
https://genius.com/Jody-watley-friends-lyrics ...
https://genius.com/Ray-parker-jr-ghostbusters-lyrics ...
https://genius.com/Elvis-presley-good-luck-charm-lyrics ...
https://genius.com/Johnny-nash-i-can-see-clearly-now-lyrics ...
https://genius.com/Barry-white-im-gonna-love-you-just-a-little-more-baby-lyrics ...
https://genius.com/Yvonne-elliman-if-i-cant-have-you-lyrics ...
https://genius.com/Chakachas-jungle-fever-lyrics ...
https://genius.com/Reo-speedwagon-keep-on-loving-you-lyrics ...
https://genius.com/Labelle-lady-marmalade-lyrics ...
h

0      [(Now And Then There’s) A Fool Such As I Lyric...
19     [TranslationsFrançaisBreaking Free Lyrics[Intr...
21     [Can’t Fight This Feeling Lyrics[Verse 1]\nOh,...
24     [Can’t we try - single version LyricsI see you...
26     [Drive Lyrics[Verse 1]\nWho's gonna tell you w...
45     [Dreams Lyrics[Intro]\n\n[Verse 1]\nNow here y...
52     [For What It’s Worth Lyrics[Verse 1]\nThere's ...
54     [Friends Lyrics[Verse 1: Jody Watley]\nHave yo...
56     [Ghostbusters Lyrics(Ghostbusters)\nIf there's...
58     [Good Luck Charm Lyrics[Verse 1]\nDon't want a...
69     [I Can See Clearly Now Lyrics[Verse 1]\nI can ...
84     [I’m Gonna Love You Just A Little More, Baby L...
88     [If I Can’t Have You Lyrics[Verse 1]\nDon't kn...
95     [Jungle Fever Lyrics[Instrumental]\n\n[Female]...
97     [Keep on Loving You Lyrics[Verse 1]\nYou shoul...
99     [Lady Marmalade Lyrics[Intro]\nHey sister, go ...
110    [Mickey Lyrics[Refrain]\nOh, Mickey\nYou're so...
121    [Not Tonight Lyrics[Vers

In [238]:
better_url_refetches_form_400_DONE = save_lyrics(
    convert_fetched_lyrics_to_df(
        res,
        _temp
    ),
    'TEMP these are the refetches for urls that differ, hand corrected, within the 400 misc refetches'
)

In [239]:
better_url_refetches_form_400_DONE

,song_id,lyrics,urls
0,(Now and Then There's) A Fool Such As IElvis P...,(Now And Then There’s) A Fool Such As I Lyrics...,"[""https://genius.com/Elvis-presley-now-and-the..."
19,"Breaking FreeZac Efron, Andrew Seeley & Vaness...",TranslationsFrançaisBreaking Free Lyrics[Intro...,"[""https://genius.com/High-school-musical-cast-..."
21,Can't Fight This FeelingREO Speedwagon,"Can’t Fight This Feeling Lyrics[Verse 1]\nOh, ...","[""https://genius.com/Reo-speedwagon-cant-fight..."
24,Can't We TryDan Hill (Duet With Vonda Shepard),Can’t we try - single version LyricsI see your...,"[""https://genius.com/Dan-hill-cant-we-try-sing..."
26,CarsGary Numan,Drive Lyrics[Verse 1]\nWho's gonna tell you wh...,"[""https://genius.com/The-cars-drive-lyrics""]"
45,DreamsFleetwood Mac,Dreams Lyrics[Intro]\n\n[Verse 1]\nNow here yo...,"[""https://genius.com/Fleetwood-mac-dreams-lyri..."
52,"For What It's Worth (Stop, Hey What's That Sou...",For What It’s Worth Lyrics[Verse 1]\nThere's s...,"[""https://genius.com/Buffalo-springfield-for-w..."
54,FriendsJody Watley With Eric B. & Rakim,Friends Lyrics[Verse 1: Jody Watley]\nHave you...,"[""https://genius.com/Jody-watley-friends-lyrics""]"
56,GhostbustersRay Parker Jr.,Ghostbusters Lyrics(Ghostbusters)\nIf there's ...,"[""https://genius.com/Ray-parker-jr-ghostbuster..."
58,Good Luck CharmElvis Presley With The Jordanaires,Good Luck Charm Lyrics[Verse 1]\nDon't want a ...,"[""https://genius.com/Elvis-presley-good-luck-c..."


### 2. These failed to get *any* URLS, so obivously they don't have a `likely_correct_url`. ACTION: JUST REFETCH, then HAND CODE anything that *still* fails 😭 

#### 2a. refetch failed!

In [128]:
_try_refetching = _fetched_just_name_df[
    _fetched_just_name_df.lyrics.isna()
]

_res = fetch_lyrics_and_urls(
    _try_refetching
)

Mike Reno And Ann Wilson Almost Paradise...Love Theme From Footloose
CLEANED:  Mike Reno And Ann Wilson Almost ParadiseLove Theme From Footloose
[]
OOPS: Mike Reno And Ann Wilson Almost Paradise...Love Theme From Footloose
Patti Austin With James Ingram Baby, Come To Me
CLEANED:  Patti Austin With James Ingram Baby Come To Me
[]
OOPS: Patti Austin With James Ingram Baby, Come To Me
Elton John Candle In The Wind 1997/Something About The Way You Look Tonight
CLEANED:  Elton John Candle In The Wind 1997Something About The Way You Look Tonight
[]
OOPS: Elton John Candle In The Wind 1997/Something About The Way You Look Tonight
Elvis Presley Don't Cry Daddy/Rubberneckin'
CLEANED:  Elvis Presley Dont Cry DaddyRubberneckin
[]
OOPS: Elvis Presley Don't Cry Daddy/Rubberneckin'
Creedence Clearwater Revival Down On The Corner/Fortunate Son
CLEANED:  Creedence Clearwater Revival Down On The CornerFortunate Son
[]
OOPS: Creedence Clearwater Revival Down On The Corner/Fortunate Son
Olivia Rodrigo Dr

[]
OOPS: Kenny Loggins Nobody's Fool (Theme From "Caddyshack II")
Brook Benton Rainy Night In Georgia/Rubberneckin'
CLEANED:  Brook Benton Rainy Night In GeorgiaRubberneckin
[]
OOPS: Brook Benton Rainy Night In Georgia/Rubberneckin'
Iyaz Replay
CLEANED:  Iyaz Replay
['https://genius.com/Iyaz-replay-lyrics', 'https://genius.com/Iyaz-replay-remix-lyrics', 'https://genius.com/Iyaz-replay-donni-hotwheel-mixshow-lyrics', 'https://genius.com/Hoodie-trash-replay-hoodie-trash-remix-lyrics', 'https://genius.com/Angeldust666-replay-lyrics', 'https://genius.com/Formerly-vinylzombie-mcr-nostalgic-songs-annotated', 'https://genius.com/Bure-0-100-freestyle-lyrics', 'https://genius.com/Figueroa-run-it-back-lyrics', 'https://genius.com/Lil-gherkin-weekday-lyrics', 'https://genius.com/Scott-the-pisces-perfect-summer-lyrics']
Nickelback Rockstar
CLEANED:  Nickelback Rockstar
['https://genius.com/Nickelback-rockstar-lyrics', 'https://genius.com/Nickelback-rockstar-sea-shanty-lyrics', 'https://genius.com/

In [133]:
third_refetch_sigh = convert_fetched_lyrics_to_df(
    _res,
    _try_refetching
)
third_refetch_sigh

,song_id,lyrics,urls
9,Almost Paradise...Love Theme From FootlooseMik...,None,None
15,"Baby, Come To MePatti Austin With James Ingram",None,None
25,Candle In The Wind 1997/Something About The Wa...,None,None
39,Don't Cry Daddy/Rubberneckin'Elvis Presley,None,None
43,Down On The Corner/Fortunate SonCreedence Clea...,None,None
46,Drivers LicenseOlivia Rodrigo,TranslationsTürkçeEspañolРусскийPortuguêsPortu...,"[""https://genius.com/Olivia-rodrigo-drivers-li..."
64,Hotline BlingDrake,TranslationsTürkçeEspañolPortuguêsFrançaisHotl...,"[""https://genius.com/Drake-hotline-bling-lyric..."
65,How Am I Supposed To Live Without YouMichael B...,How Am I Supposed to Live Without You Lyrics[V...,"[""https://genius.com/Michael-bolton-how-am-i-s..."
66,How You Remind MeNickelback,How You Remind Me Lyrics[Verse 1]\nNever made ...,"[""https://genius.com/Nickelback-how-you-remind..."
76,I Ran (So Far Away)A Flock Of Seagulls,TranslationsTürkçeI Ran (So Far Away) Lyrics[I...,"[""https://genius.com/A-flock-of-seagulls-i-ran..."


#### 2b. Hand code failed ones

In [137]:
third_refetch_sigh_df = third_refetch_sigh[
        third_refetch_sigh.lyrics.isna()
    ].merge(right=songs_df, on='song_id', how='left')

In [175]:
# NOTE: ok, the following songs fail cuz they were actually *2* songs billed as *1*, and so lyric genius has them separate
# REASON: songs had a "b side" or "flip side", that shipped with the main song (often but not always less of a hit)
# TEMP FIX: I've just chosen lyrics for the FIRST song listed
# 'Candle In The Wind 1997/Something About The Way You Look TonightElton John'
# "Don't Cry Daddy/Rubberneckin'Elvis Presley"

HAND_SET_URLS = {
    'Almost Paradise...Love Theme From FootlooseMike Reno And Ann Wilson': 'https://genius.com/Mike-reno-and-ann-wilson-almost-paradise-lyrics',
    'Baby, Come To MePatti Austin With James Ingram': 'https://genius.com/Patti-austin-baby-come-to-me-lyrics',
    'Candle In The Wind 1997/Something About The Way You Look TonightElton John': 'https://genius.com/Elton-john-something-about-the-way-you-look-tonight-lyrics',
    "Don't Cry Daddy/Rubberneckin'Elvis Presley": 'https://genius.com/Elvis-presley-dont-cry-daddy-lyrics',
    'Down On The Corner/Fortunate SonCreedence Clearwater Revival':'https://genius.com/Creedence-clearwater-revival-down-on-the-corner-lyrics',
     "I'm So Lonesome I Could CryB.J. Thomas And The Triumphs": 'https://genius.com/Bj-thomas-im-so-lonesome-i-could-cry-lyrics',
    'Nobody\'s Fool (Theme From "Caddyshack II")Kenny Loggins': "https://genius.com/Kenny-loggins-nobodys-fool-lyrics",
    "Rainy Night In Georgia/Rubberneckin'Brook Benton": 'https://genius.com/Brook-benton-rainy-night-in-georgia-lyrics',
    'Saving Forever For You (From "Beverly Hills, 90210")Shanice': 'https://genius.com/Shanice-saving-forever-for-you-lyrics',
    'Shakedown (From "Beverly Hills Cop II")Bob Seger': 'https://genius.com/Bob-seger-shakedown-lyrics',
    'Superstar/Bless The Beasts And ChildrenCarpenters': 'https://genius.com/Carpenters-superstar-lyrics',
    "Travelin' Band/Who'll Stop The RainCreedence Clearwater Revival": 'https://genius.com/Creedence-clearwater-revival-travelin-band-lyrics',
    'pride.is.the.devilJ. Cole & Lil Baby': 'https://genius.com/J-cole-and-lil-baby-p-r-i-d-e-i-s-t-h-e-d-e-v-i-l-lyrics',
    
}

In [177]:
def search_song_chose_first_then_get_lyrics(row):
    try: 
        url = HAND_SET_URLS[row['song_id']]
        return [genius.lyrics(song_url=url), [url]]
    except:
        url = HAND_SET_URLS[row['song_id']]
        print(f'OOPS: {url}')
        
def __fetch_lyrics(songs_subset_df):
    return songs_subset_df.apply(
        lambda row: search_song_chose_first_then_get_lyrics(row),
        axis=1
    )

__hand_urls_fetched = __fetch_lyrics(third_refetch_sigh_df)

In [181]:
hand_fetched_df = convert_fetched_lyrics_to_df(
    __hand_urls_fetched,
    third_refetch_sigh_df
)
hand_fetched_df

,song_id,lyrics,urls
0,Almost Paradise...Love Theme From FootlooseMik...,Almost Paradise Lyrics[Verse 1:]\nI thought th...,"[""https://genius.com/Mike-reno-and-ann-wilson-..."
1,"Baby, Come To MePatti Austin With James Ingram","Baby, Come to Me Lyrics[Verse 1: Patti Austin]...","[""https://genius.com/Patti-austin-baby-come-to..."
2,Candle In The Wind 1997/Something About The Wa...,Something About the Way You Look Tonight Lyric...,"[""https://genius.com/Elton-john-something-abou..."
3,Don't Cry Daddy/Rubberneckin'Elvis Presley,Don’t Cry Daddy Lyrics[Verse 1]\nToday I stumb...,"[""https://genius.com/Elvis-presley-dont-cry-da..."
4,Down On The Corner/Fortunate SonCreedence Clea...,Down on the Corner Lyrics[Verse 1]\nEarly in t...,"[""https://genius.com/Creedence-clearwater-revi..."
5,I'm So Lonesome I Could CryB.J. Thomas And The...,I’m So Lonesome I Could Cry LyricsDid you hear...,"[""https://genius.com/Bj-thomas-im-so-lonesome-..."
6,"Nobody's Fool (Theme From ""Caddyshack II"")Kenn...",Nobody’s Fool Lyrics[Verse 1]\nI know you thin...,"[""https://genius.com/Kenny-loggins-nobodys-foo..."
7,Rainy Night In Georgia/Rubberneckin'Brook Benton,Rainy Night in Georgia Lyrics[Verse 1]\nHoveri...,"[""https://genius.com/Brook-benton-rainy-night-..."
8,"Saving Forever For You (From ""Beverly Hills, 9...",Saving Forever For You Lyrics[Verse 1]\nI've n...,"[""https://genius.com/Shanice-saving-forever-fo..."
9,"Shakedown (From ""Beverly Hills Cop II"")Bob Seger",Shakedown Lyrics[Verse 1]\nNo matter what you ...,"[""https://genius.com/Bob-seger-shakedown-lyrics""]"


#### 2c. AT LONG LAST... MWAHAHAHA AHAHAHA.. I SHALL STITCH THE *FAILED* value together 

In [204]:
# 2a
third_refetch_sigh.dropna(subset=['lyrics'])
# 2b
hand_fetched_df

# 2c
_failed_to_fetch_so_refetched_and_hand_set_df = pd.concat([third_refetch_sigh.dropna(subset=['lyrics']), hand_fetched_df])

save_lyrics(_failed_to_fetch_so_refetched_and_hand_set_df, 'TEMP TO BE SAFE _failed_to_fetch_so_refetched_and_hand_set_df from 400')


,song_id,lyrics,urls
46,Drivers LicenseOlivia Rodrigo,TranslationsTürkçeEspañolРусскийPortuguêsPortu...,"[""https://genius.com/Olivia-rodrigo-drivers-li..."
64,Hotline BlingDrake,TranslationsTürkçeEspañolPortuguêsFrançaisHotl...,"[""https://genius.com/Drake-hotline-bling-lyric..."
65,How Am I Supposed To Live Without YouMichael B...,How Am I Supposed to Live Without You Lyrics[V...,"[""https://genius.com/Michael-bolton-how-am-i-s..."
66,How You Remind MeNickelback,How You Remind Me Lyrics[Verse 1]\nNever made ...,"[""https://genius.com/Nickelback-how-you-remind..."
76,I Ran (So Far Away)A Flock Of Seagulls,TranslationsTürkçeI Ran (So Far Away) Lyrics[I...,"[""https://genius.com/A-flock-of-seagulls-i-ran..."
78,I Will Always Love YouWhitney Houston,TranslationsΕλληνικάI Will Always Love You Lyr...,"[""https://genius.com/Whitney-houston-i-will-al..."
81,I'd Do Anything For Love (But I Won't Do That)...,I’d Do Anything for Love (But I Won’t Do That)...,"[""https://genius.com/Meat-loaf-id-do-anything-..."
114,Ms. JacksonOutKast,"Ms. Jackson Lyrics[Intro: André 3000]\nYeah, t...","[""https://genius.com/Outkast-ms-jackson-lyrics..."
118,No AirJordin Sparks Duet With Chris Brown,No Air Lyrics[Intro: Jordin Sparks]\nTell me h...,"[""https://genius.com/Jordin-sparks-no-air-lyri..."
136,ReplayIyaz,Replay Lyrics[Chorus]\nShawty's like a melody ...,"[""https://genius.com/Iyaz-replay-lyrics"", ""htt..."


# OK, SO let's see how good we fixed this:

## THE ISSUE WE'RE FIXING: there are 250 rows in the refetched 423 without lyrics

In [285]:
big_refetch_of_misc

,song_id,lyrics,urls
0,(Now and Then There's) A Fool Such As IElvis P...,NaN,NaN
1,(Sittin' On) The Dock Of The BayOtis Redding,(Sittin’ On) The Dock of the Bay Lyrics[Verse ...,"[""https://genius.com/Otis-redding-sittin-on-th..."
2,(You Gotta) Fight For Your Right (To Party!)Be...,Fight for Your Right Lyrics[Intro]\nYeah\nKick...,"[""https://genius.com/Beastie-boys-fight-for-yo..."
3,(You're the) Devil In DisguiseElvis Presley Wi...,NaN,NaN
4,...Baby One More TimeBritney Spears,TranslationsItaliano...Baby One More Time Lyri...,"[""https://genius.com/Britney-spears-baby-one-m..."
...,...,...,...
418,"You're The First, The Last, My EverythingBarry...",NaN,NaN
419,Youngblood5 Seconds Of Summer,TranslationsEspañolPortuguêsYoungblood Lyrics[...,"[""https://genius.com/5-seconds-of-summer-young..."
420,Your LoveThe Outfield,NaN,NaN
421,YummyJustin Bieber,NaN,NaN


In [287]:
big_refetch_of_misc[
    big_refetch_of_misc.lyrics.isna()
]

,song_id,lyrics,urls
0,(Now and Then There's) A Fool Such As IElvis P...,NaN,NaN
3,(You're the) Devil In DisguiseElvis Presley Wi...,NaN,NaN
8,95.southJ. Cole,NaN,"[""https://genius.com/8-bit-arcade-95-south-8-b..."
9,A Big Hunk O' LoveElvis Presley With The Jorda...,NaN,NaN
10,A Fifth Of BeethovenWalter Murphy & The Big Ap...,NaN,"[""https://genius.com/Walter-murphy-a-fifth-of-..."
...,...,...,...
415,You Turn Me On (Turn On Song)Ian Whitcomb And ...,NaN,NaN
418,"You're The First, The Last, My EverythingBarry...",NaN,NaN
420,Your LoveThe Outfield,NaN,NaN
421,YummyJustin Bieber,NaN,NaN


## A. Removing instrumentals, we see only 192 lyrics are *truly* missing

In [292]:
'95.southJ. Cole' in instrumental_formatted_as_df.song_id.values

True

In [293]:
instrumental_formatted_as_df = pd.read_csv(f'{LYRIC_FOLDER_PATH}/{"INSTRUMENTAL SONGS final"}.csv')

_instramentals_removed = big_refetch_of_misc[
    big_refetch_of_misc.apply(lambda row: row['song_id'] not in instrumental_formatted_as_df.song_id.values, axis=1)
]
_instramentals_removed.lyrics.isna().sum()

192

## B. Let's create a non-duplicative version of the other fetches to concat: 195 rows

In [296]:
__non_dup_fetched_just_name =  _fetched_just_name_df.dropna(subset=['lyrics'])[
    _fetched_just_name_df.dropna(subset=['lyrics']).apply(
        lambda row: row['song_id'] not in _failed_to_fetch_so_refetched_and_hand_set_df.song_id.values, 
        axis=1
    )
]

__all_refetched_lyrics = pd.concat([
    __non_dup_fetched_just_name[['song_id', 'lyrics', 'urls']],
    _failed_to_fetch_so_refetched_and_hand_set_df
])
__all_refetched_lyrics

,song_id,lyrics,urls
0,(Now and Then There's) A Fool Such As IElvis P...,"(Now And Then, There’s) A Fool Such As I Lyric...","[""https://genius.com/Hank-snow-now-and-then-th..."
1,(You're the) Devil In DisguiseElvis Presley Wi...,(You’re The) Devil In Disguise Lyrics[Pre-Chor...,"[""https://genius.com/Elvis-presley-youre-the-d..."
2,A Big Hunk O' LoveElvis Presley With The Jorda...,"A Big Hunk O’ Love Lyrics[Intro]\nHey baby, I ...","[""https://genius.com/Elvis-presley-a-big-hunk-..."
3,A Horse With No NameAmerica,A Horse With No Name Lyrics[Verse 1]\nOn the f...,"[""https://genius.com/America-a-horse-with-no-n..."
4,A Million To OneJimmy Charles and The Revelletts,"A Million to One Lyrics[Verse 1]\nBaby, now th...","[""https://genius.com/Kiss-a-million-to-one-lyr..."
...,...,...,...
8,"Saving Forever For You (From ""Beverly Hills, 9...",Saving Forever For You Lyrics[Verse 1]\nI've n...,"[""https://genius.com/Shanice-saving-forever-fo..."
9,"Shakedown (From ""Beverly Hills Cop II"")Bob Seger",Shakedown Lyrics[Verse 1]\nNo matter what you ...,"[""https://genius.com/Bob-seger-shakedown-lyrics""]"
10,Superstar/Bless The Beasts And ChildrenCarpenters,Superstar Lyrics[Verse 1]\nLong ago and oh so ...,"[""https://genius.com/Carpenters-superstar-lyri..."
11,Travelin' Band/Who'll Stop The RainCreedence C...,Travelin’ Band Lyrics[Verse 1]\n737 coming out...,"[""https://genius.com/Creedence-clearwater-revi..."


## C. Let's now add those refetches to the main dataset, assuming the refetched ones are more accurate

In [299]:
__all_refetched_lyrics
_instramentals_removed


__all_fetched_lyrics_MERGED_TOGETHER_YAY = pd.concat([
    _instramentals_removed.dropna(subset=['lyrics'])[
        _instramentals_removed.dropna(subset=['lyrics']).apply(
            lambda row: row['song_id'] not in __all_refetched_lyrics.song_id.values, 
            axis=1
        )
    ],
    __all_refetched_lyrics
])
__all_fetched_lyrics_MERGED_TOGETHER_YAY

,song_id,lyrics,urls
1,(Sittin' On) The Dock Of The BayOtis Redding,(Sittin’ On) The Dock of the Bay Lyrics[Verse ...,"[""https://genius.com/Otis-redding-sittin-on-th..."
2,(You Gotta) Fight For Your Right (To Party!)Be...,Fight for Your Right Lyrics[Intro]\nYeah\nKick...,"[""https://genius.com/Beastie-boys-fight-for-yo..."
4,...Baby One More TimeBritney Spears,TranslationsItaliano...Baby One More Time Lyri...,"[""https://genius.com/Britney-spears-baby-one-m..."
5,24K MagicBruno Mars,TranslationsPortuguês24K Magic Lyrics[Intro]\n...,"[""https://genius.com/Bruno-mars-24k-magic-lyri..."
6,25 Or 6 To 4Chicago,25 or 6 to 4 Lyrics[Verse 1: Peter Cetera]\nWa...,"[""https://genius.com/Chicago-25-or-6-to-4-lyri..."
...,...,...,...
8,"Saving Forever For You (From ""Beverly Hills, 9...",Saving Forever For You Lyrics[Verse 1]\nI've n...,"[""https://genius.com/Shanice-saving-forever-fo..."
9,"Shakedown (From ""Beverly Hills Cop II"")Bob Seger",Shakedown Lyrics[Verse 1]\nNo matter what you ...,"[""https://genius.com/Bob-seger-shakedown-lyrics""]"
10,Superstar/Bless The Beasts And ChildrenCarpenters,Superstar Lyrics[Verse 1]\nLong ago and oh so ...,"[""https://genius.com/Carpenters-superstar-lyri..."
11,Travelin' Band/Who'll Stop The RainCreedence C...,Travelin’ Band Lyrics[Verse 1]\n737 coming out...,"[""https://genius.com/Creedence-clearwater-revi..."


### Ok, what did we miss? HOLY FUCK NOTHING WUT WAIT ... ARE WE DONE??????!?!?!?!?!?! 😳 

In [309]:
_instramentals_removed[
    _instramentals_removed.song_id.apply(
        lambda v: v not in __all_fetched_lyrics_MERGED_TOGETHER_YAY.song_id.values,
    )
]

,song_id,lyrics,urls


## EXPORT THIS MUTHATRUCKER!!!

In [313]:
save_lyrics(
    __all_fetched_lyrics_MERGED_TOGETHER_YAY,
    'DONE WOOHOO refetching about 420 non-lyric-ed songs and removing instrumentals'
)

,song_id,lyrics,urls
1,(Sittin' On) The Dock Of The BayOtis Redding,(Sittin’ On) The Dock of the Bay Lyrics[Verse ...,"[""https://genius.com/Otis-redding-sittin-on-th..."
2,(You Gotta) Fight For Your Right (To Party!)Be...,Fight for Your Right Lyrics[Intro]\nYeah\nKick...,"[""https://genius.com/Beastie-boys-fight-for-yo..."
4,...Baby One More TimeBritney Spears,TranslationsItaliano...Baby One More Time Lyri...,"[""https://genius.com/Britney-spears-baby-one-m..."
5,24K MagicBruno Mars,TranslationsPortuguês24K Magic Lyrics[Intro]\n...,"[""https://genius.com/Bruno-mars-24k-magic-lyri..."
6,25 Or 6 To 4Chicago,25 or 6 to 4 Lyrics[Verse 1: Peter Cetera]\nWa...,"[""https://genius.com/Chicago-25-or-6-to-4-lyri..."
...,...,...,...
8,"Saving Forever For You (From ""Beverly Hills, 9...",Saving Forever For You Lyrics[Verse 1]\nI've n...,"[""https://genius.com/Shanice-saving-forever-fo..."
9,"Shakedown (From ""Beverly Hills Cop II"")Bob Seger",Shakedown Lyrics[Verse 1]\nNo matter what you ...,"[""https://genius.com/Bob-seger-shakedown-lyrics""]"
10,Superstar/Bless The Beasts And ChildrenCarpenters,Superstar Lyrics[Verse 1]\nLong ago and oh so ...,"[""https://genius.com/Carpenters-superstar-lyri..."
11,Travelin' Band/Who'll Stop The RainCreedence C...,Travelin’ Band Lyrics[Verse 1]\n737 coming out...,"[""https://genius.com/Creedence-clearwater-revi..."


# NEXT ACTION, DAVID: pull all ⬆️ these datasets together, save them as one big "400 misc" dataset to use in notebook 2.6

### SEPERATE OUT INSTRUMENTALS 🎺 

In [242]:
IS_INSTRUMENTAL_RETURN_FLAG = 'THIS_IS_AN_INSTRUMENTAL'

In [243]:
_failed_refetch = big_refetch_of_misc[
    big_refetch_of_misc.lyrics.isna()
    & big_refetch_of_misc.urls.notna()
]
_failed_refetch

,song_id,lyrics,urls
8,95.southJ. Cole,NaN,"[""https://genius.com/8-bit-arcade-95-south-8-b..."
10,A Fifth Of BeethovenWalter Murphy & The Big Ap...,NaN,"[""https://genius.com/Walter-murphy-a-fifth-of-..."
27,Axel FHarold Faltermeyer,NaN,"[""https://genius.com/Harold-faltermeyer-axel-f..."
54,Can't Hold UsMacklemore & Ryan Lewis Featuring...,NaN,"[""https://genius.com/Cant-hold-us-cant-hold-us..."
63,Classical GasMason Williams,NaN,"[""https://genius.com/Mason-williams-classical-..."
79,Disco Duck (Part I)Rick Dees & His Cast Of Idiots,NaN,"[""https://genius.com/Rick-dees-and-his-cast-of..."
105,ExodusFerrante & Teicher,NaN,"[""https://genius.com/Ferrante-and-teicher-exod..."
108,Feels So GoodChuck Mangione,NaN,"[""https://genius.com/Chuck-mangione-feels-so-g..."
121,Grazing In The GrassHugh Masekela,NaN,"[""https://genius.com/Hugh-masekela-grazing-in-..."
122,Green OnionsBooker T. & The MG's,NaN,"[""https://genius.com/Booker-t-and-the-mgs-gree..."


In [244]:
_merged_with = songs_df.merge(left_on='song_id', right=_failed_refetch, right_on='song_id')

In [19]:
# this SHOULDN'T TAKE LONG
re_refetched_to_see_instrumentals = fetch_lyrics_and_urls(
    _merged_with
)
re_refetched_to_see_instrumentals

J. Cole 95.south
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/8-bit-arcade-95-south-8-bit-j-cole-emulation-lyrics
Walter Murphy & The Big Apple Band A Fifth Of Beethoven
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Walter-murphy-a-fifth-of-beethoven-lyrics
Harold Faltermeyer Axel F
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Harold-faltermeyer-axel-f-lyrics
Macklemore & Ryan Lewis Featuring Ray Dalton Can't Hold Us
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Cant-hold-us-cant-hold-us-tribute-to-macklemore-ryan-lewis-and-ray-dalton-instrumental-version-lyrics
Mason Williams Classical Gas
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song 

Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Taylor-dayne-tell-it-to-my-heart-dub-of-hearts-mix-lyrics
The Tornadoes Telstar
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/The-tornados-telstar-lyrics
Ramsey Lewis Trio The "In" Crowd
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/The-ramsey-lewis-trio-the-in-crowd-lyrics
Dave "Baby" Cortez The Happy Organ
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Dave-baby-cortez-the-happy-organ-lyrics
Cliff Nobles & Co. The Horse
Hi, I edited this.
Couldn't find the lyrics section. Please report this if the song has lyrics.
Song URL: https://genius.com/Cliff-nobles-and-co-the-horse-lyrics
Herb Alpert & The Tijuana Brass The Lonely Bull (El

0     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
1     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
2     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
3     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
4     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
5     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
6     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
7     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
8     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
9     [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
10    [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
11    [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
12    [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
13    [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
14    [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
15    [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
16    [THIS_IS_AN_INSTRUMENTAL, [https://genius.com/...
17    [THIS_IS_AN_INSTRUMENTAL, [https://genius.

In [245]:
# Caught this when looking at lyric length
INSTRUMENTAL_TO_APPEND = {
    'song_id': "Only YouFranck Pourcel's French Fiddles",
    'lyrics': IS_INSTRUMENTAL_RETURN_FLAG,
    'urls': np.nan, 
}

instrumental_formatted_as_df = convert_fetched_lyrics_to_df(
    re_refetched_to_see_instrumentals,
    _merged_with
).append(INSTRUMENTAL_TO_APPEND, ignore_index = True)
instrumental_formatted_as_df

NameError: name 're_refetched_to_see_instrumentals' is not defined

In [35]:
save_lyrics(
    instrumental_formatted_as_df,
    'all the INSTRUMENTAL songs'
)

,song_id,lyrics,urls
0,95.southJ. Cole,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/8-bit-arcade-95-south-8-b..."
1,A Fifth Of BeethovenWalter Murphy & The Big Ap...,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Walter-murphy-a-fifth-of-..."
2,Axel FHarold Faltermeyer,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Harold-faltermeyer-axel-f..."
3,Can't Hold UsMacklemore & Ryan Lewis Featuring...,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Cant-hold-us-cant-hold-us..."
4,Classical GasMason Williams,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Mason-williams-classical-..."
5,Disco Duck (Part I)Rick Dees & His Cast Of Idiots,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Rick-dees-and-his-cast-of..."
6,ExodusFerrante & Teicher,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Ferrante-and-teicher-exod..."
7,Feels So GoodChuck Mangione,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Chuck-mangione-feels-so-g..."
8,Grazing In The GrassHugh Masekela,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Hugh-masekela-grazing-in-..."
9,Green OnionsBooker T. & The MG's,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Booker-t-and-the-mgs-gree..."


In [248]:

instrumental_formatted_as_df = pd.read_csv(f'{LYRIC_FOLDER_PATH}/{"all the INSTRUMENTAL songs"}.csv')

In [256]:
# add additionally found ones:
instrumental_formatted_as_df_FINAL = instrumental_formatted_as_df.copy()
for instrumental_id in EVEN_MORE_INSTRUMENTALS_FOUND:
    instrumental_formatted_as_df_FINAL = instrumental_formatted_as_df_FINAL.append(
        {
            'song_id': instrumental_id,
            'lyrics': IS_INSTRUMENTAL_RETURN_FLAG,
            'urls': np.nan, 
        },
        ignore_index=True
    )

save_lyrics(
    instrumental_formatted_as_df_FINAL.drop_duplicates(subset=['song_id']),
    'INSTRUMENTAL SONGS final'
)


,song_id,lyrics,urls
0,95.southJ. Cole,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/8-bit-arcade-95-south-8-b..."
1,A Fifth Of BeethovenWalter Murphy & The Big Ap...,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Walter-murphy-a-fifth-of-..."
2,Axel FHarold Faltermeyer,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Harold-faltermeyer-axel-f..."
3,Can't Hold UsMacklemore & Ryan Lewis Featuring...,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Cant-hold-us-cant-hold-us..."
4,Classical GasMason Williams,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Mason-williams-classical-..."
5,Disco Duck (Part I)Rick Dees & His Cast Of Idiots,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Rick-dees-and-his-cast-of..."
6,ExodusFerrante & Teicher,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Ferrante-and-teicher-exod..."
7,Feels So GoodChuck Mangione,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Chuck-mangione-feels-so-g..."
8,Grazing In The GrassHugh Masekela,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Hugh-masekela-grazing-in-..."
9,Green OnionsBooker T. & The MG's,THIS_IS_AN_INSTRUMENTAL,"[""https://genius.com/Booker-t-and-the-mgs-gree..."


_____________________________

# OK, SO EVERYTHING 👇 IS WORK I DID REFETCHING DATA. Everything 👆 is work just processing that data, saving it to disk.

# 1a. Has lyrics! Yay! Done.

In [69]:
# has_lyrics = pd.concat([
#     second_stashed_fetched_lyrics_with_url, stashed_fetched_lyrics_with_url
# ]).dropna(subset=['lyrics']).drop_duplicates(subset=['song_id'])

# save_lyrics(has_lyrics, 'has_lyrics')

In [73]:
has_lyrics_df = pd.read_csv(f'./data/lyric-fetching-datasets/{"has_lyrics"}.csv')
has_lyrics_df

,song_id,lyrics,urls
0,(I Can't Get No) SatisfactionThe Rolling Stones,(I Can’t Get No) Satisfaction Lyrics[Instrumen...,"[""https://genius.com/The-rolling-stones-i-cant..."
1,(I Just) Died In Your ArmsCutting Crew,(I Just) Died in Your Arms Lyrics[Intro]\nOh I...,"[""https://genius.com/Cutting-crew-i-just-died-..."
2,(I Know) I'm Losing YouThe Temptations,(I Know) I’m Losing You Lyrics[Intro]\nOoh\nYo...,"[""https://genius.com/The-temptations-i-know-im..."
3,(I've Had) The Time Of My LifeBill Medley & Je...,(I’ve Had) The Time of My Life Lyrics[Chorus: ...,"[""https://genius.com/Bill-medley-and-jennifer-..."
4,1 ThingAmerie,1 Thing Lyrics[Intro]\nWoo!\nUh\nWoo!\nNa-na-n...,"[""https://genius.com/Amerie-1-thing-lyrics"", ""..."
...,...,...,...
4638,abcdefuGAYLE,TranslationsTürkçeEspañolDeutschFrançais​abcde...,"[""https://genius.com/Gayle-abcdefu-lyrics"", ""h..."
4639,amariJ. Cole,"American Dream Lyrics[Intro: Jeezy]\n(Cryin', ...","[""https://genius.com/Jeezy-american-dream-lyri..."
4640,iSpyKYLE Featuring Lil Yachty,"I Spy, T Shirt, Isn’t She Lovely, & Swang Mash...","[""https://genius.com/Alex-aiono-i-spy-t-shirt-..."
4641,interludeJ. Cole,"Interlude (Cole World) LyricsI'm drivin', nigg...","[""https://genius.com/J-cole-interlude-cole-wor..."


In [123]:
merged_with_lyrics = pd.merge(
    left=songs_df, 
    left_on='song_id', 
    right=has_lyrics_df, 
    right_on='song_id',
    how='left' # keep all songs
)

merged_with_lyrics['fetched_url'] =  merged_with_lyrics.urls.apply(
    lambda stringified_array: pd.notna(stringified_array) and json.loads(stringified_array)[0]
)
merged_with_lyrics['urls'] =  merged_with_lyrics.urls.apply(
    lambda stringified_array: pd.notna(stringified_array) and json.loads(stringified_array)
)
merged_with_lyrics

,song_id,chart_position,chart_debut,song,performer,artist_array,generic_genre,lyrics,urls,fetched_url
0,#9 DreamJohn Lennon,9,1974-12-21,#9 Dream,John Lennon,['John Lennon'],uncategorized,#9 Dream Lyrics[Verse 1]\nSo long ago\nWas it ...,[https://genius.com/John-lennon-9-dream-lyrics...,https://genius.com/John-lennon-9-dream-lyrics
1,'03 Bonnie & ClydeJay-Z Featuring Beyonce Knowles,4,2002-10-26,'03 Bonnie & Clyde,Jay-Z Featuring Beyonce Knowles,"['Jay-Z', 'Beyonce Knowles']",rock,"’03 Bonnie & Clyde Lyrics[Intro: JAY-Z]\nUhh, ...",[https://genius.com/Jay-z-03-bonnie-and-clyde-...,https://genius.com/Jay-z-03-bonnie-and-clyde-l...
2,'65 Love AffairPaul Davis,6,1982-02-27,'65 Love Affair,Paul Davis,['Paul Davis'],hip hop,’65 Love Affair LyricsI was a car hop\nYou wer...,[https://genius.com/Paul-davis-65-love-affair-...,https://genius.com/Paul-davis-65-love-affair-l...
3,('til) I Kissed YouThe Everly Brothers,4,1959-08-15,('til) I Kissed You,The Everly Brothers,['The Everly Brothers'],country,(’Til) I Kissed You Lyrics[Verse]\nNever felt ...,[https://genius.com/The-everly-brothers-til-i-...,https://genius.com/The-everly-brothers-til-i-k...
4,(Can't Live Without Your) Love And AffectionNe...,1,1990-07-07,(Can't Live Without Your) Love And Affection,Nelson,['Nelson'],uncategorized,(Can’t Live Without Your) Love And Affection L...,[https://genius.com/Nelson-cant-live-without-y...,https://genius.com/Nelson-cant-live-without-yo...
...,...,...,...,...,...,...,...,...,...,...
5061,everything i wantedBillie Eilish,8,2019-11-23,everything i wanted,Billie Eilish,['Billie Eilish'],folk,TranslationsTürkçeBillie Eilish - everything i...,[https://genius.com/Billie-eilish-everything-i...,https://genius.com/Billie-eilish-everything-i-...
5062,iSpyKYLE Featuring Lil Yachty,4,2017-01-14,iSpy,KYLE Featuring Lil Yachty,"['KYLE', 'Lil Yachty']",pop,"I Spy, T Shirt, Isn’t She Lovely, & Swang Mash...",[https://genius.com/Alex-aiono-i-spy-t-shirt-i...,https://genius.com/Alex-aiono-i-spy-t-shirt-is...
5063,interludeJ. Cole,8,2021-05-22,interlude,J. Cole,['J. Cole'],NaN,"Interlude (Cole World) LyricsI'm drivin', nigg...",[https://genius.com/J-cole-interlude-cole-worl...,https://genius.com/J-cole-interlude-cole-world...
5064,"my.lifeJ. Cole, 21 Savage & Morray",2,2021-05-29,my.life,"J. Cole, 21 Savage & Morray","['J. Cole', '21 Savage', 'Morray']",NaN,TranslationsРусскийPortuguêsDeutschItaliano​​m...,[https://genius.com/J-cole-21-savage-and-morra...,https://genius.com/J-cole-21-savage-and-morray...


# 1b. Has lyrics... but they're (probalby) wrong. Fix!

### They don't end in `-lyrics` as most correct URLs do...

In [178]:
url_does_not_end_with_lyrics = merged_with_lyrics[
    merged_with_lyrics.lyrics.notna() &
    (merged_with_lyrics.lyrics != '[]') &
    ~merged_with_lyrics.fetched_url.apply(lambda url: url and url.endswith('-lyrics'))
]

url_does_not_end_with_lyrics

,song_id,chart_position,chart_debut,song,performer,artist_array,generic_genre,lyrics,urls,fetched_url
42,(You're) Having My BabyPaul Anka with Odia Coates,1,1974-07-06,(You're) Having My Baby,Paul Anka with Odia Coates,['Paul Anka with Odia Coates'],hip hop,Angela’s Ashes LyricsAngela's Ashes\n\nA Memoi...,[https://genius.com/Frank-mccourt-angelas-ashe...,https://genius.com/Frank-mccourt-angelas-ashes...
55,1-800-273-8255Logic Featuring Alessia Cara & K...,3,2017-05-20,1-800-273-8255,Logic Featuring Alessia Cara & Khalid,"['Logic', 'Alessia Cara', 'Khalid']",NaN,"Favorite Cover Songs Lyrics-#-- ""1-800-273-825...",[https://genius.com/Goodapolloiv-favorite-cove...,https://genius.com/Goodapolloiv-favorite-cover...
70,34+35Ariana Grande Feat. Doja Cat & Megan Thee...,2,2021-01-02,34+35,Ariana Grande Feat. Doja Cat & Megan Thee Stal...,"['Ariana Grande Feat. Doja Cat', 'Megan Thee S...",uncategorized,Doja Cat [Discography List] LyricsStudio Album...,[https://genius.com/Doja-cat-doja-cat-discogra...,https://genius.com/Doja-cat-doja-cat-discograp...
77,6 Foot 7 FootLil Wayne Featuring Cory Gunz,9,2011-01-01,6 Foot 7 Foot,Lil Wayne Featuring Cory Gunz,"['Lil Wayne', 'Cory Gunz']",uncategorized,Best lines/bars of the 2010's Lyrics Englishha...,[https://genius.com/Genius-users-best-lines-ba...,https://genius.com/Genius-users-best-lines-bar...
78,679Fetty Wap Featuring Remy Boyz,4,2015-07-18,679,Fetty Wap Featuring Remy Boyz,"['Fetty Wap', 'Remy Boyz']",pop,New Music Tuesday 06/30/15 LyricsRefused - Ser...,[https://genius.com/Spotify-new-music-tuesday-...,https://genius.com/Spotify-new-music-tuesday-0...
...,...,...,...,...,...,...,...,...,...,...
4937,You Are So Beautiful/It's A Sin When You Love ...,5,1975-01-11,You Are So Beautiful/It's A Sin When You Love ...,Joe Cocker,['Joe Cocker'],country,Angela’s Ashes LyricsAngela's Ashes\n\nA Memoi...,[https://genius.com/Frank-mccourt-angelas-ashe...,https://genius.com/Frank-mccourt-angelas-ashes...
4977,You Learn/You Oughta KnowAlanis Morissette,6,1996-07-27,You Learn/You Oughta Know,Alanis Morissette,['Alanis Morissette'],pop,Spotify Singles: Complete Collection LyricsThe...,[https://genius.com/Spotify-spotify-singles-co...,https://genius.com/Spotify-spotify-singles-com...
5016,You're Makin' Me High/Let It FlowToni Braxton,1,1996-06-08,You're Makin' Me High/Let It Flow,Toni Braxton,['Toni Braxton'],NaN,Toni Braxton’s 25 Best Songs: Critic’s Picks L...,[https://genius.com/Billboard-toni-braxtons-25...,https://genius.com/Billboard-toni-braxtons-25-...
5040,Young ForeverJay-Z + Mr. Hudson,10,2009-09-26,Young Forever,Jay-Z + Mr. Hudson,['Jay-Z + Mr. Hudson'],country,Die liebsten Closing-Tracks der Community Lyri...,[https://genius.com/Genius-deutschland-die-lie...,https://genius.com/Genius-deutschland-die-lieb...


### ... they literally *all* end in `-annotated` instead, LOL. Found a consistent bug!

In [177]:
url_does_not_end_with_lyrics.fetched_url.apply(lambda s: s[-10:]).describe()

count            245
unique             1
top       -annotated
freq             245
Name: fetched_url, dtype: object

## Ok, so we're going to just refetch the lyrics using the `url` that closely matches the tiltle (if it exists), as measured by `Levenshtein distance` (fancy!)

In [36]:
from Levenshtein import distance

In [37]:
a ='Lil Wayne Featuring Cory Gunz'  
s = '6 Foot 7 Foot'
u = 'https://genius.com/Lil-wayne-6-foot-7-foot-lyrics'
def get_l_dist(performer, song, url):
    format_song = f'{performer.split()[0]} {song} lyrics'
    format_url = url.replace('https://genius.com/', '').replace('-', ' ')
    return distance(
        format_song,
        format_url
    )
# (format_song, format_url)

In [38]:
def return_likely_correct_url_or_else_false(row):
    for url in row['urls']:
        if url.endswith('-lyrics'): 
            dist = get_l_dist(row['performer'], row['song'], url)
            if dist < 13:
                return url
    return np.nan

In [195]:
url_does_not_end_with_lyrics.apply(return_likely_correct_url_or_else_false, axis=1)

42                                                    NaN
55                                                    NaN
70                                                    NaN
77      https://genius.com/Lil-wayne-6-foot-7-foot-lyrics
78                https://genius.com/Fetty-wap-679-lyrics
                              ...                        
4937                                                  NaN
4977                                                  NaN
5016                                                  NaN
5040        https://genius.com/Jay-z-young-forever-lyrics
5047    https://genius.com/Kesha-your-love-is-my-drug-...
Length: 245, dtype: object

In [210]:
with_likely_correct_url = url_does_not_end_with_lyrics.copy()
with_likely_correct_url['likely_correct_url'] = url_does_not_end_with_lyrics.apply(
    return_likely_correct_url_or_else_false, axis=1
)

with_likely_correct_url = with_likely_correct_url.dropna(subset=['likely_correct_url'])

def _get_song(url, urls):
    try: 
        print(url, '...')
        return [genius.lyrics(song_url=url), urls]
    except:
        print(f'OOPS: {url}')
        
def refetch_lyrics_and_urls(songs_subset_df):
    return songs_subset_df.apply(
        lambda row: _get_song(row['likely_correct_url'], row['urls']),
        axis=1
    )

refeched = with_likely_correct_url.apply(
        lambda row: _get_song(row['likely_correct_url'], row['urls']),
        axis=1
    )

https://genius.com/Lil-wayne-6-foot-7-foot-lyrics ...
https://genius.com/Fetty-wap-679-lyrics ...
https://genius.com/Timbaland-apologize-lyrics ...
https://genius.com/Migos-bad-and-boujee-lyrics ...
https://genius.com/Travie-mccoy-billionaire-lyrics ...
https://genius.com/Rae-sremmurd-black-beatles-lyrics ...
https://genius.com/Robin-thicke-blurred-lines-lyrics ...
https://genius.com/Bts-boy-with-luv-lyrics ...
https://genius.com/Katy-perry-california-gurls-lyrics ...
https://genius.com/Fabolous-cant-let-you-go-lyrics ...
https://genius.com/Flo-rida-club-cant-handle-me-lyrics ...
https://genius.com/Snoop-dogg-drop-it-like-its-hot-lyrics ...
OOPS: https://genius.com/Snoop-dogg-drop-it-like-its-hot-lyrics
https://genius.com/Jay-z-empire-state-of-mind-lyrics ...
https://genius.com/6ix9ine-fefe-lyrics ...
https://genius.com/Drake-fair-trade-lyrics ...
https://genius.com/Pitbull-feel-this-moment-lyrics ...
https://genius.com/Bruno-mars-finesse-remix-lyrics ...
https://genius.com/Irene-cara-

In [211]:
refeched

77      [TranslationsEnglish6 Foot 7 Foot Lyrics[Choru...
78      [679 Lyrics[Produced by Peoples]\n\n[Intro: Fe...
280     [Apologize Lyrics[Intro: Timbaland]\nAyy, ayy,...
347     [TranslationsPortuguêsРусскийItalianoAzərbayca...
458     [Billionaire Lyrics[Pre-Chorus: Bruno Mars]\nI...
                              ...                        
4648    [We Are Young Lyrics[Verse 1: Nate Ruess]\nGiv...
4664    [TranslationsPortuguêsEnglishFrançaisWe Found ...
4670    [TranslationsPortuguêsWe R Who We R Lyrics[Ver...
5040    [Young Forever Lyrics[Intro: Mr. Hudson]\nLet'...
5047    [TranslationsPortuguêsYour Love is My Drug Lyr...
Length: 69, dtype: object

# TODO: update `fetched_url` with `likely_correct_url` used in these calls

In [214]:
def convert_fetched_lyrics_to_df(fetched_lyrics_and_url_options_arrays, original_df):
    just_lyrics = [x and x[0] for x in fetched_lyrics_and_url_options_arrays]
    just_stringified_urls = [x and json.dumps(x[1]) for x in fetched_lyrics_and_url_options_arrays]
    save_to_disk = pd.DataFrame({
        'song_id': original_df.song_id, 'lyrics': just_lyrics, 'urls': just_stringified_urls, 
        'likely_correct_url': original_df.likely_correct_url
    })
    return save_to_disk

In [216]:
refetched_from_likely_df = convert_fetched_lyrics_to_df(refeched, with_likely_correct_url)

In [218]:
save_lyrics(refetched_from_likely_df, 'TODO we refetched approx 70 songs with a more likely URL, integrate them')

,song_id,lyrics,urls,likely_correct_url
77,6 Foot 7 FootLil Wayne Featuring Cory Gunz,TranslationsEnglish6 Foot 7 Foot Lyrics[Chorus...,"[""https://genius.com/Genius-users-best-lines-b...",https://genius.com/Lil-wayne-6-foot-7-foot-lyrics
78,679Fetty Wap Featuring Remy Boyz,679 Lyrics[Produced by Peoples]\n\n[Intro: Fet...,"[""https://genius.com/Spotify-new-music-tuesday...",https://genius.com/Fetty-wap-679-lyrics
280,ApologizeTimbaland Featuring OneRepublic,"Apologize Lyrics[Intro: Timbaland]\nAyy, ayy, ...","[""https://genius.com/Nvp-summer-playlist-2015-...",https://genius.com/Timbaland-apologize-lyrics
347,Bad And BoujeeMigos Featuring Lil Uzi Vert,TranslationsPortuguêsРусскийItalianoAzərbaycan...,"[""https://genius.com/Miguel-castillo-top-5-jam...",https://genius.com/Migos-bad-and-boujee-lyrics
458,BillionaireTravie McCoy Featuring Bruno Mars,Billionaire Lyrics[Pre-Chorus: Bruno Mars]\nI ...,"[""https://genius.com/Formerly-vinylzombie-mcr-...",https://genius.com/Travie-mccoy-billionaire-ly...
...,...,...,...,...
4648,We Are Youngfun. Featuring Janelle Monae,We Are Young Lyrics[Verse 1: Nate Ruess]\nGive...,"[""https://genius.com/Adoseofbuckley-the-ten-wo...",https://genius.com/Fun-we-are-young-lyrics
4664,We Found LoveRihanna Featuring Calvin Harris,TranslationsPortuguêsEnglishFrançaisWe Found L...,"[""https://genius.com/Npr-the-200-greatest-song...",https://genius.com/Rihanna-we-found-love-lyrics
4670,We R Who We RKe$ha,TranslationsPortuguêsWe R Who We R Lyrics[Vers...,"[""https://genius.com/Adoseofbuckley-adoseofbuc...",https://genius.com/Kesha-we-r-who-we-r-lyrics
5040,Young ForeverJay-Z + Mr. Hudson,Young Forever Lyrics[Intro: Mr. Hudson]\nLet's...,"[""https://genius.com/Genius-deutschland-die-li...",https://genius.com/Jay-z-young-forever-lyrics


# 2. ⚠️ IGNORE FOR NOW ⚠️  Has urls (not lyrics) -- try refetching!

#  3. Needs to be refetched

In [86]:
merged_with_lyrics[
    merged_with_lyrics.lyrics.isna()
]

,song_id,chart_position,chart_debut,song,performer,artist_array,generic_genre,lyrics,urls,fetched_url
25,(Now and Then There's) A Fool Such As IElvis P...,2,1959-03-21,(Now and Then There's) A Fool Such As I,Elvis Presley With The Jordanaires,['Elvis Presley With The Jordanaires'],pop,NaN,NaN,False
29,(Sittin' On) The Dock Of The BayOtis Redding,1,1968-01-27,(Sittin' On) The Dock Of The Bay,Otis Redding,['Otis Redding'],pop,NaN,NaN,False
39,(You Gotta) Fight For Your Right (To Party!)Be...,7,1986-12-20,(You Gotta) Fight For Your Right (To Party!),Beastie Boys,['Beastie Boys'],pop,NaN,NaN,False
41,(You're the) Devil In DisguiseElvis Presley Wi...,3,1963-06-29,(You're the) Devil In Disguise,Elvis Presley With The Jordanaires,['Elvis Presley With The Jordanaires'],pop,NaN,NaN,False
47,...Baby One More TimeBritney Spears,1,1998-11-21,...Baby One More Time,Britney Spears,['Britney Spears'],hip hop,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...
5026,"You're The First, The Last, My EverythingBarry...",2,1974-11-02,"You're The First, The Last, My Everything",Barry White,['Barry White'],pop,NaN,NaN,False
5046,Youngblood5 Seconds Of Summer,7,2018-06-16,Youngblood,5 Seconds Of Summer,['5 Seconds Of Summer'],rock,NaN,NaN,False
5048,Your LoveThe Outfield,6,1986-02-15,Your Love,The Outfield,['The Outfield'],NaN,NaN,NaN,False
5056,YummyJustin Bieber,2,2020-01-18,Yummy,Justin Bieber,['Justin Bieber'],country,NaN,NaN,False
